In [ ]:
#This code drags and drops shapes 
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QFrame, QVBoxLayout, QWidget, QHBoxLayout
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QDrag
from PyQt5.QtCore import Qt, QMimeData, QPoint
import subprocess
import time
import rospy
import roslaunch
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class ShapeLabel(QLabel):
    def __init__(self, shape, parent=None):
        super(ShapeLabel, self).__init__(parent)
        self.shape = shape  #circle or square
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setText(shape)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event): #Checks if the left mouse button is pressed.
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : lightgray; }")
        self.shapes = [] #Initializes an empty list to store dropped shapes and their positions.

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():  #Checks if the mime data contains text.
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):  #Gets the shape text and position from the event.
        shape = event.mimeData().text()
        position = event.pos()
        self.shapes.append((shape, position))
        self.update()
        event.acceptProposedAction()

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)
        
        painter.end()

class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Initialize ROS node
        rospy.init_node('rpl_line_gui', anonymous=True)
        self.bridge = CvBridge()

        # Launch files
        uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
        roslaunch.configure_logging(uuid)
        self.launch = roslaunch.parent.ROSLaunchParent(uuid, [
            "/home/nmis/Documents/INTERNSHIP/GUI/RPL-GUI-V.1-main/launch/param_server_cell.launch",
        ])
        self.launch.start()

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Subscribe to camera topics
        self.image_sub1 = rospy.Subscriber('/camera1/image_raw', Image, self.callback_cam1)
        self.image_sub2 = rospy.Subscriber('/camera2/image_raw', Image, self.callback_cam2)
        self.image_sub3 = rospy.Subscriber('/camera3/image_raw', Image, self.callback_cam3)

        # Add drag-and-drop shapes to the sidebar
        self.circleLabel = ShapeLabel("Circle", self)
        self.squareLabel = ShapeLabel("Square", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleLabel)
        self.shapeLayout.addWidget(self.squareLabel)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.shapeContainer)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Add a drop area to the center
        self.dropArea = DropArea(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)

        self.centralwidget.setLayout(self.mainLayout)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def callback_cam1(self, data):
        self.display_image(data, self.sensorView1)

    def callback_cam2(self, data):
        self.display_image(data, self.sensorView2)
    
    def callback_cam3(self, data):
        self.display_image(data, self.sensorView3)
    
    def display_image(self, data, label):
        cv_image = self.bridge.imgmsg_to_cv2(data, "bgr8")
        qt_image = QImage(cv_image.data, cv_image.shape[1], cv_image.shape[0], cv_image.strides[0], QImage.Format_RGB888)
        label.setPixmap(QPixmap.fromImage(qt_image))

    def closeEvent(self, event):
        self.launch.shutdown()
        if self.roscore is not None:
            self.roscore.terminate()
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    sys.exit(app.exec_())


In [ ]:
import sys
import numpy as np
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QFrame, QVBoxLayout, QWidget, QHBoxLayout
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QDrag, QPolygonF
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF
import subprocess
import time
import rospy
import roslaunch
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class ShapeLabel(QLabel):
    def __init__(self, shape, parent=None):
        super(ShapeLabel, self).__init__(parent)
        self.shape = shape  # Circle or Square
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setText(shape)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class EraserLabel(QLabel):
    def __init__(self, parent=None):
        super(EraserLabel, self).__init__(parent)
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setText("Eraser")
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText("Eraser")

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap.scaled(self.width() // 2, self.height() // 2, Qt.KeepAspectRatio))
            drag.setHotSpot(event.pos() // 2)

            drag.exec_(Qt.MoveAction)

class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : lightgray; }")
        self.shapes = []  # Initializes an empty list to store dropped shapes and their positions.
        self.connections = []  # List to store connections between shapes
        self.last_right_clicked = None  # Store the last right-clicked shape

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        if event.mimeData().text() == "Eraser":
            # Eraser drop handling will be implemented later
            pass
        else:
            shape = event.mimeData().text()
            position = event.pos()
            self.shapes.append((shape, position))
        self.update()
        event.acceptProposedAction()

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

        for connection in self.connections:
            start, end = connection
            self.draw_arrow(painter, start, end)
        
        painter.end()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = event.pos()
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_pos = self.last_right_clicked[1]
                    end_pos = clicked_shape[1]
                    self.connections.append((start_pos, end_pos))
                    self.last_right_clicked = None
                    self.update()
        super(DropArea, self).mousePressEvent(event)

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None

    def draw_arrow(self, painter, start, end):
        line = QLineF(start, end)
        painter.drawLine(line)

        # Calculate arrow head
        arrow_size = 10
        angle = np.arctan2(-(end.y() - start.y()), end.x() - start.x())
        p1 = end + QPointF(np.cos(angle + np.pi / 3) * arrow_size,
                           np.sin(angle + np.pi / 3) * arrow_size)
        p2 = end + QPointF(np.cos(angle - np.pi / 3) * arrow_size,
                           np.sin(angle - np.pi / 3) * arrow_size)

        arrow_head = QPolygonF([end, p1, p2])
        painter.setBrush(Qt.black)
        painter.drawPolygon(arrow_head)

class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Initialize ROS node
        rospy.init_node('rpl_line_gui', anonymous=True)
        self.bridge = CvBridge()

        # Launch files
        uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
        roslaunch.configure_logging(uuid)
        self.launch = roslaunch.parent.ROSLaunchParent(uuid, [
            "/home/nmis/Documents/INTERNSHIP/GUI/RPL-GUI-V.1-main/launch/param_server_cell.launch",
        ])
        self.launch.start()

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Subscribe to camera topics
        self.image_sub1 = rospy.Subscriber('/camera1/image_raw', Image, self.callback_cam1)
        self.image_sub2 = rospy.Subscriber('/camera2/image_raw', Image, self.callback_cam2)
        self.image_sub3 = rospy.Subscriber('/camera3/image_raw', Image, self.callback_cam3)

        # Add drag-and-drop shapes to the sidebar
        self.circleLabel = ShapeLabel("Circle", self)
        self.squareLabel = ShapeLabel("Square", self)
        self.eraserLabel = EraserLabel(self)

        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleLabel)
        self.shapeLayout.addWidget(self.squareLabel)
        self.shapeLayout.addWidget(self.eraserLabel)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.shapeContainer)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Add a drop area to the center
        self.dropArea = DropArea(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)

        self.centralwidget.setLayout(self.mainLayout)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def callback_cam1(self, data):
        self.display_image(data, self.sensorView1)

    def callback_cam2(self, data):
        self.display_image(data, self.sensorView2)
    
    def callback_cam3(self, data):
        self.display_image(data, self.sensorView3)
    
    def display_image(self, data, label):
        cv_image = self.bridge.imgmsg_to_cv2(data, "bgr8")
        qt_image = QImage(cv_image.data, cv_image.shape[1], cv_image.shape[0], cv_image.strides[0], QImage.Format_RGB888)
        label.setPixmap(QPixmap.fromImage(qt_image))

    def closeEvent(self, event):
        self.launch.shutdown()
        if self.roscore is not None:
            self.roscore.terminate()
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    sys.exit(app.exec_())


In [ ]:
#This code LINKS and Arrows the nodes
import sys
import numpy as np
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QFrame, QVBoxLayout, QWidget, QHBoxLayout
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QDrag, QPolygonF
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF
import subprocess
import time
import rospy
import roslaunch
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class ShapeLabel(QLabel):
    def __init__(self, shape, parent=None):
        super(ShapeLabel, self).__init__(parent)
        self.shape = shape  # Circle or Square
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setText(shape)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : lightgray; }")
        self.shapes = []  # Initializes an empty list to store dropped shapes and their positions.
        self.connections = []  # List to store connections between shapes
        self.last_right_clicked = None  # Store the last right-clicked shape

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        shape = event.mimeData().text()
        position = event.pos()
        self.shapes.append((shape, position))
        self.update()
        event.acceptProposedAction()

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

        for connection in self.connections:
            start, end = connection
            self.draw_arrow(painter, start, end)
        
        painter.end()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = event.pos()
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_pos = self.last_right_clicked[1]
                    end_pos = clicked_shape[1]
                    self.connections.append((start_pos, end_pos))
                    self.last_right_clicked = None
                    self.update()
        super(DropArea, self).mousePressEvent(event)

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None

    def draw_arrow(self, painter, start, end):
        line = QLineF(start, end)
        painter.drawLine(line)

        # Calculate arrow head
        arrow_size = 10
        angle = np.arctan2(-(end.y() - start.y()), end.x() - start.x())
        p1 = end + QPointF(np.cos(angle + np.pi / 3) * arrow_size,
                           np.sin(angle + np.pi / 3) * arrow_size)
        p2 = end + QPointF(np.cos(angle - np.pi / 3) * arrow_size,
                           np.sin(angle - np.pi / 3) * arrow_size)

        arrow_head = QPolygonF([end, p1, p2])
        painter.setBrush(Qt.black)
        painter.drawPolygon(arrow_head)

class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Initialize ROS node
        rospy.init_node('rpl_line_gui', anonymous=True)
        self.bridge = CvBridge()

        # Launch files
        uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
        roslaunch.configure_logging(uuid)
        self.launch = roslaunch.parent.ROSLaunchParent(uuid, [
            "/home/nmis/Documents/INTERNSHIP/GUI/RPL-GUI-V.1-main/launch/param_server_cell.launch",
        ])
        self.launch.start()

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Subscribe to camera topics
        self.image_sub1 = rospy.Subscriber('/camera1/image_raw', Image, self.callback_cam1)
        self.image_sub2 = rospy.Subscriber('/camera2/image_raw', Image, self.callback_cam2)
        self.image_sub3 = rospy.Subscriber('/camera3/image_raw', Image, self.callback_cam3)

        # Add drag-and-drop shapes to the sidebar
        self.circleLabel = ShapeLabel("Circle", self)
        self.squareLabel = ShapeLabel("Square", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleLabel)
        self.shapeLayout.addWidget(self.squareLabel)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.shapeContainer)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Add a drop area to the center
        self.dropArea = DropArea(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)

        self.centralwidget.setLayout(self.mainLayout)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def callback_cam1(self, data):
        self.display_image(data, self.sensorView1)

    def callback_cam2(self, data):
        self.display_image(data, self.sensorView2)
    
    def callback_cam3(self, data):
        self.display_image(data, self.sensorView3)
    
    def display_image(self, data, label):
        cv_image = self.bridge.imgmsg_to_cv2(data, "bgr8")
        qt_image = QImage(cv_image.data, cv_image.shape[1], cv_image.shape[0], cv_image.strides[0], QImage.Format_RGB888)
        label.setPixmap(QPixmap.fromImage(qt_image))

    def closeEvent(self, event):
        self.launch.shutdown()
        if self.roscore is not None:
            self.roscore.terminate()
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    sys.exit(app.exec_())


In [ ]:
#this code drags eraser and deleted the link(s)
import sys
import numpy as np
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QFrame, QVBoxLayout, QWidget, QHBoxLayout
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QDrag, QPolygonF
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF
import subprocess
import time
import rospy
import roslaunch
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class ShapeLabel(QLabel):
    def __init__(self, shape, parent=None):
        super(ShapeLabel, self).__init__(parent)
        self.shape = shape  # Circle or Square
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setText(shape)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class EraserLabel(QLabel):
    def __init__(self, parent=None):
        super(EraserLabel, self).__init__(parent)
        self.setText("Eraser")
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText("Eraser")

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : lightgray; }")
        self.shapes = []  # Initializes an empty list to store dropped shapes and their positions.
        self.connections = []  # List to store connections between shapes
        self.last_right_clicked = None  # Store the last right-clicked shape
        

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        if event.mimeData().text() == "Eraser":
            self.erase_connection(event.pos())
        else:
            shape = event.mimeData().text()
            position = event.pos()
            self.shapes.append((shape, position))
        self.update()
        event.acceptProposedAction()

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

        for connection in self.connections:
            start, end = connection
            self.draw_arrow(painter, start, end)
        
        painter.end()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = event.pos()
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_pos = self.last_right_clicked[1]
                    end_pos = clicked_shape[1]
                    self.connections.append((start_pos, end_pos))
                    self.last_right_clicked = None
                    self.update()
        super(DropArea, self).mousePressEvent(event)

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None

    def draw_arrow(self, painter, start, end):
        line = QLineF(start, end)
        painter.drawLine(line)

        # Calculate arrow head
        arrow_size = 10
        angle = np.arctan2(-(end.y() - start.y()), end.x() - start.x())
        p1 = end + QPointF(np.cos(angle + np.pi / 3) * arrow_size,
                           np.sin(angle + np.pi / 3) * arrow_size)
        p2 = end + QPointF(np.cos(angle - np.pi / 3) * arrow_size,
                           np.sin(angle - np.pi / 3) * arrow_size)

        arrow_head = QPolygonF([end, p1, p2])
        painter.setBrush(Qt.black)
        painter.drawPolygon(arrow_head)

    def erase_connection(self, pos):
        # Find and remove the connection closest to the given position
        for connection in self.connections:
            start, end = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    break

class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Initialize ROS node
        rospy.init_node('rpl_line_gui', anonymous=True)
        self.bridge = CvBridge()

        # Launch files
        uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
        roslaunch.configure_logging(uuid)
        self.launch = roslaunch.parent.ROSLaunchParent(uuid, [
            "/home/nmis/Documents/INTERNSHIP/GUI/RPL-GUI-V.1-main/launch/param_server_cell.launch",
        ])
        self.launch.start()

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Subscribe to camera topics
        self.image_sub1 = rospy.Subscriber('/camera1/image_raw', Image, self.callback_cam1)
        self.image_sub2 = rospy.Subscriber('/camera2/image_raw', Image, self.callback_cam2)
        self.image_sub3 = rospy.Subscriber('/camera3/image_raw', Image, self.callback_cam3)

        # Add drag-and-drop shapes to the sidebar
        self.circleLabel = ShapeLabel("Circle", self)
        self.squareLabel = ShapeLabel("Square", self)
        self.eraserLabel = EraserLabel(self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleLabel)
        self.shapeLayout.addWidget(self.squareLabel)
        self.shapeLayout.addWidget(self.eraserLabel)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.shapeContainer)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Add a drop area to the center
        self.dropArea = DropArea(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)

        self.centralwidget.setLayout(self.mainLayout)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass


    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def callback_cam1(self, data):
        self.display_image(data, self.sensorView1)

    def callback_cam2(self, data):
        self.display_image(data, self.sensorView2)
    
    def callback_cam3(self, data):
        self.display_image(data, self.sensorView3)
    
    def display_image(self, data, label):
        cv_image = self.bridge.imgmsg_to_cv2(data, "bgr8")
        qt_image = QImage(cv_image.data, cv_image.shape[1], cv_image.shape[0], cv_image.strides[0], QImage.Format_RGB888)
        label.setPixmap(QPixmap.fromImage(qt_image))

    def closeEvent(self, event):
        self.launch.shutdown()
        if self.roscore is not None:
            self.roscore.terminate()
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    sys.exit(app.exec_())


In [ ]:
#this code can delete both links and shapes 
import sys
import numpy as np
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QFrame, QVBoxLayout, QWidget, QHBoxLayout
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QDrag, QPolygonF
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF
import subprocess
import time
import rospy
import roslaunch
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class ShapeLabel(QLabel):
    def __init__(self, shape, parent=None):
        super(ShapeLabel, self).__init__(parent)
        self.shape = shape  # Circle or Square
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setText(shape)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class EraserLabel(QLabel):
    def __init__(self, parent=None):
        super(EraserLabel, self).__init__(parent)
        self.setText("Eraser")
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText("Eraser")

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            # Reduce size of eraser during drag
            pixmap = QPixmap(self.size() * 0.5)
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos() * 0.5)

            drag.exec_(Qt.MoveAction)

class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : lightgray; }")
        self.shapes = []  # Initializes an empty list to store dropped shapes and their positions.
        self.connections = []  # List to store connections between shapes
        self.last_right_clicked = None  # Store the last right-clicked shape

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        if event.mimeData().text() == "Eraser":
            self.erase_item(event.pos())
        else:
            shape = event.mimeData().text()
            position = event.pos()
            self.shapes.append((shape, position))
        self.update()
        event.acceptProposedAction()

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

        for connection in self.connections:
            start, end = connection
            self.draw_arrow(painter, start, end)
        
        painter.end()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = event.pos()
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_pos = self.last_right_clicked[1]
                    end_pos = clicked_shape[1]
                    self.connections.append((start_pos, end_pos))
                    self.last_right_clicked = None
                    self.update()
        super(DropArea, self).mousePressEvent(event)

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None

    def draw_arrow(self, painter, start, end):
        line = QLineF(start, end)
        painter.drawLine(line)

        # Calculate arrow head
        arrow_size = 10
        angle = np.arctan2(-(end.y() - start.y()), end.x() - start.x())
        p1 = end + QPointF(np.cos(angle + np.pi / 3) * arrow_size,
                           np.sin(angle + np.pi / 3) * arrow_size)
        p2 = end + QPointF(np.cos(angle - np.pi / 3) * arrow_size,
                           np.sin(angle - np.pi / 3) * arrow_size)

        arrow_head = QPolygonF([end, p1, p2])
        painter.setBrush(Qt.black)
        painter.drawPolygon(arrow_head)

    def erase_item(self, pos):
        self.erase_shape(pos)
        self.erase_connection(pos)

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end = connection
            line = QLineF(start, end)
            if self.is_point_near_line(pos, line):
                self.connections.remove(connection)
                break

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
               (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25):
                self.shapes.remove((shape, position))
                break

    def is_point_near_line(self, point, line, threshold=5):
        p1 = line.p1()
        p2 = line.p2()
        distance = np.abs((p2.y() - p1.y()) * point.x() - (p2.x() - p1.x()) * point.y() + p2.x() * p1.y() - p2.y() * p1.x()) / np.sqrt((p2.y() - p1.y())**2 + (p2.x() - p1.x())**2)
        return distance < threshold
    
class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Initialize ROS node
        rospy.init_node('rpl_line_gui', anonymous=True)
        self.bridge = CvBridge()

        # Launch files
        uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
        roslaunch.configure_logging(uuid)
        self.launch = roslaunch.parent.ROSLaunchParent(uuid, [
            "/home/nmis/Documents/INTERNSHIP/GUI/RPL-GUI-V.1-main/launch/param_server_cell.launch",
        ])
        self.launch.start()

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Subscribe to camera topics
        self.image_sub1 = rospy.Subscriber('/camera1/image_raw', Image, self.callback_cam1)
        self.image_sub2 = rospy.Subscriber('/camera2/image_raw', Image, self.callback_cam2)
        self.image_sub3 = rospy.Subscriber('/camera3/image_raw', Image, self.callback_cam3)

        # Add drag-and-drop shapes to the sidebar
        self.circleLabel = ShapeLabel("Circle", self)
        self.squareLabel = ShapeLabel("Square", self)
        self.eraserLabel = EraserLabel(self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleLabel)
        self.shapeLayout.addWidget(self.squareLabel)
        self.shapeLayout.addWidget(self.eraserLabel)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.shapeContainer)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Add a drop area to the center
        self.dropArea = DropArea(self)
        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)

        self.centralwidget.setLayout(self.mainLayout)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass


    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def callback_cam1(self, data):
        self.display_image(data, self.sensorView1)

    def callback_cam2(self, data):
        self.display_image(data, self.sensorView2)
    
    def callback_cam3(self, data):
        self.display_image(data, self.sensorView3)
    
    def display_image(self, data, label):
        cv_image = self.bridge.imgmsg_to_cv2(data, "bgr8")
        qt_image = QImage(cv_image.data, cv_image.shape[1], cv_image.shape[0], cv_image.strides[0], QImage.Format_RGB888)
        label.setPixmap(QPixmap.fromImage(qt_image))

    def closeEvent(self, event):
        self.launch.shutdown()
        if self.roscore is not None:
            self.roscore.terminate()
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    sys.exit(app.exec_())


In [ ]:
#this code creates a dictionary to update the shapes 
import sys
import numpy as np
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QFrame, QVBoxLayout, QWidget, QHBoxLayout
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QDrag, QPolygonF
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF
import subprocess
import time
import rospy
import roslaunch
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class ShapeLabel(QLabel):
    def __init__(self, shape, parent=None):
        super(ShapeLabel, self).__init__(parent)
        self.shape = shape  # Circle or Square
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setText(shape)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class EraserLabel(QLabel):
    def __init__(self, parent=None):
        super(EraserLabel, self).__init__(parent)
        self.setText("Eraser")
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText("Eraser")

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)
class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : lightgray; }")
        self.shapes = []  # Initializes an empty list to store dropped shapes and their positions.
        self.connections = []  # List to store connections between shapes
        self.last_right_clicked = None  # Store the last right-clicked shape
        self.behavior_tree = {}  # Initialize the behavior tree dictionary

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        if event.mimeData().text() == "Eraser":
            self.erase_connection(event.pos())
            self.erase_shape(event.pos())
        else:
            shape = event.mimeData().text()
            position = event.pos()
            self.shapes.append((shape, position))
            
            # Add the shape to the behavior tree
            shape_id = f"{shape}_{len(self.shapes)}"
            self.behavior_tree[shape_id] = {'type': shape, 'position': (position.x(), position.y()), 'children': []}
            
        self.update()
        event.acceptProposedAction()
        
        # Print the updated behavior tree
        print("Behavior Tree:", self.behavior_tree)

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

        for connection in self.connections:
            start, end = connection
            self.draw_arrow(painter, start, end)
        
        painter.end()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = event.pos()
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.connections.append((start_pos, end_pos))
                    
                    # Update the behavior tree
                    start_shape_id = self.get_shape_id(start_shape, start_pos)
                    end_shape_id = self.get_shape_id(end_shape, end_pos)
                    if start_shape_id and end_shape_id:
                        self.behavior_tree[start_shape_id]['children'].append(end_shape_id)
                    
                    self.last_right_clicked = None
                    self.update()
                    
                    # Print the updated behavior tree
                    print("Behavior Tree:", self.behavior_tree)
        else:
            super(DropArea, self).mousePressEvent(event)

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def draw_arrow(self, painter, start, end):
        line = QLineF(start, end)
        painter.drawLine(line)

        # Calculate arrow head
        arrow_size = 10
        angle = np.arctan2(-(end.y() - start.y()), end.x() - start.x())
        p1 = end + QPointF(np.cos(angle + np.pi / 3) * arrow_size,
                           np.sin(angle + np.pi / 3) * arrow_size)
        p2 = end + QPointF(np.cos(angle - np.pi / 3) * arrow_size,
                           np.sin(angle - np.pi / 3) * arrow_size)

        arrow_head = QPolygonF([end, p1, p2])
        painter.setBrush(Qt.black)
        painter.drawPolygon(arrow_head)

    def erase_connection(self, pos):
        # Find and remove the connection closest to the given position
        for connection in self.connections:
            start, end = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    break

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
               (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25):
                self.shapes.remove((shape, position))
                break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Initialize ROS node
        rospy.init_node('rpl_line_gui', anonymous=True)
        self.bridge = CvBridge()

        # Launch files
        uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
        roslaunch.configure_logging(uuid)
        self.launch = roslaunch.parent.ROSLaunchParent(uuid, [
            "/home/nmis/Documents/INTERNSHIP/GUI/RPL-GUI-V.1-main/launch/param_server_cell.launch",
        ])
        self.launch.start()

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Subscribe to camera topics
        self.image_sub1 = rospy.Subscriber('/camera1/image_raw', Image, self.callback_cam1)
        self.image_sub2 = rospy.Subscriber('/camera2/image_raw', Image, self.callback_cam2)
        self.image_sub3 = rospy.Subscriber('/camera3/image_raw', Image, self.callback_cam3)

        # Add drag-and-drop shapes to the sidebar
        self.circleLabel = ShapeLabel("Circle", self)
        self.squareLabel = ShapeLabel("Square", self)
        self.eraserLabel = EraserLabel(self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleLabel)
        self.shapeLayout.addWidget(self.squareLabel)
        self.shapeLayout.addWidget(self.eraserLabel)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.shapeContainer)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Add a drop area to the center
        self.dropArea = DropArea(self)
        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)

        self.centralwidget.setLayout(self.mainLayout)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass


    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def callback_cam1(self, data):
        self.display_image(data, self.sensorView1)

    def callback_cam2(self, data):
        self.display_image(data, self.sensorView2)
    
    def callback_cam3(self, data):
        self.display_image(data, self.sensorView3)
    
    def display_image(self, data, label):
        cv_image = self.bridge.imgmsg_to_cv2(data, "bgr8")
        qt_image = QImage(cv_image.data, cv_image.shape[1], cv_image.shape[0], cv_image.strides[0], QImage.Format_RGB888)
        label.setPixmap(QPixmap.fromImage(qt_image))

    def closeEvent(self, event):
        self.launch.shutdown()
        if self.roscore is not None:
            self.roscore.terminate()
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    sys.exit(app.exec_())


In [ ]:
# pop up for both shapes and link. with type, robot name, robot task, descryption 
import sys
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QFrame, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit)
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QDrag, QPolygonF
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF
import subprocess
import time
import rospy
import roslaunch
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class ShapeLabel(QLabel):
    def __init__(self, shape, parent=None):
        super(ShapeLabel, self).__init__(parent)
        self.shape = shape  # Circle or Square
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setText(shape)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class EraserLabel(QLabel):
    def __init__(self, parent=None):
        super(EraserLabel, self).__init__(parent)
        self.setText("Eraser")
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText("Eraser")

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : lightgray; }")
        self.shapes = []  # Initializes an empty list to store dropped shapes and their positions.
        self.connections = []  # List to store connections between shapes
        self.last_right_clicked = None  # Store the last right-clicked shape
        self.behavior_tree = {}  # Initialize the behavior tree dictionary

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        if event.mimeData().text() == "Eraser":
            self.erase_connection(event.pos())
            self.erase_shape(event.pos())
        else:
            shape = event.mimeData().text()
            position = event.pos()
            self.shapes.append((shape, position))
            
            # Show the popup window for shape
            self.show_popup('shape')
            
            # Add the shape to the behavior tree
            shape_id = f"{shape}_{len(self.shapes)}"
            self.behavior_tree[shape_id] = {
                'type': shape, 
                'position': (position.x(), position.y()), 
                'children': [], 
                'name': self.popup_name,
                'task': self.popup_task,
                'desc': self.popup_desc
            }
            
            self.update()
            event.acceptProposedAction()

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

            # Draw the name and task beside the shape
            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                painter.drawText(position + QPointF(25, 0), f"Name: {details['name']}")
                painter.drawText(position + QPointF(25, 15), f"Task: {details['task']}")

        for connection in self.connections:
            start, end, conn_type = connection
            self.draw_arrow(painter, start, end)
            self.draw_text(painter, start, end, conn_type)
        
        painter.end()


    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = event.pos()
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup()
                    if self.popup_type:  # Only add connection if popup is saved
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        # Update the behavior tree
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        else:
            super(DropArea, self).mousePressEvent(event)

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def draw_arrow(self, painter, start, end):
        line = QLineF(start, end)
        painter.drawLine(line)

        # Calculate arrow head
        arrow_size = 10
        angle = np.arctan2(-(end.y() - start.y()), end.x() - start.x())
        p1 = end + QPointF(np.cos(angle + np.pi / 3) * arrow_size,
                           np.sin(angle + np.pi / 3) * arrow_size)
        p2 = end + QPointF(np.cos(angle - np.pi / 3) * arrow_size,
                           np.sin(angle - np.pi / 3) * arrow_size)

        arrow_head = QPolygonF([end, p1, p2])
        painter.setBrush(Qt.black)
        painter.drawPolygon(arrow_head)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def erase_connection(self, pos):
        # Find and remove the connection closest to the given position
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    break

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
               (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25):
                self.shapes.remove((shape, position))
                break

    def show_popup(self, popup_type='link'):
        dialog = PopupDialog(self, popup_type)
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
            print("Saved")
            
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None
            print("Cancelled")




class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Initialize ROS node
        rospy.init_node('rpl_line_gui', anonymous=True)
        self.bridge = CvBridge()

        # Launch files
        uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
        roslaunch.configure_logging(uuid)
        self.launch = roslaunch.parent.ROSLaunchParent(uuid, [
            "/home/nmis/Documents/INTERNSHIP/GUI/RPL-GUI-V.1-main/launch/param_server_cell.launch",
        ])
        self.launch.start()

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Subscribe to camera topics
        self.image_sub1 = rospy.Subscriber('/camera1/image_raw', Image, self.callback_cam1)
        self.image_sub2 = rospy.Subscriber('/camera2/image_raw', Image, self.callback_cam2)
        self.image_sub3 = rospy.Subscriber('/camera3/image_raw', Image, self.callback_cam3)

        # Add drag-and-drop shapes to the sidebar
        self.circleLabel = ShapeLabel("Circle", self)
        self.squareLabel = ShapeLabel("Square", self)
        self.eraserLabel = EraserLabel(self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleLabel)
        self.shapeLayout.addWidget(self.squareLabel)
        self.shapeLayout.addWidget(self.eraserLabel)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.shapeContainer)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Add a drop area to the center
        self.dropArea = DropArea(self)
        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)

        self.centralwidget.setLayout(self.mainLayout)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass


    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def callback_cam1(self, data):
        self.display_image(data, self.sensorView1)

    def callback_cam2(self, data):
        self.display_image(data, self.sensorView2)
    
    def callback_cam3(self, data):
        self.display_image(data, self.sensorView3)
    
    def display_image(self, data, label):
        cv_image = self.bridge.imgmsg_to_cv2(data, "bgr8")
        qt_image = QImage(cv_image.data, cv_image.shape[1], cv_image.shape[0], cv_image.strides[0], QImage.Format_RGB888)
        label.setPixmap(QPixmap.fromImage(qt_image))

    def closeEvent(self, event):
        self.launch.shutdown()
        if self.roscore is not None:
            self.roscore.terminate()
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    sys.exit(app.exec_())


In [ ]:
#pixelated version, ALL DONE
import sys
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QFrame, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit)
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QDrag, QPolygonF
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF
import subprocess
import time
import cv2
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class ShapeLabel(QLabel):
    def __init__(self, shape, parent=None):
        super(ShapeLabel, self).__init__(parent)
        self.shape = shape  # Circle or Square
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setText(shape)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class EraserLabel(QLabel):
    def __init__(self, parent=None):
        super(EraserLabel, self).__init__(parent)
        self.setText("Eraser")
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText("Eraser")

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)




class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : lightgray; }")
        self.shapes = []  # Initializes an empty list to store dropped shapes and their positions.
        self.connections = []  # List to store connections between shapes
        self.last_right_clicked = None  # Store the last right-clicked shape
        self.behavior_tree = {}  # Initialize the behavior tree dictionary

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = event.pos()
        if event.mimeData().text() == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            shape = event.mimeData().text()
            self.shapes.append((shape, position))
            shape_id = f"{shape}_{len(self.shapes)}"
            self.show_popup('shape')
            if self.popup_name is not None:
                self.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': [],
                    'name': self.popup_name,
                    'task': self.popup_task,
                    'desc': self.popup_desc
                }
            self.update()
            event.acceptProposedAction()

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    # Remove all connections involving this shape
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
            print("Saved")
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None
            print("Cancelled")

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

            # Draw the robot name and task
            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(position.x() + 25, position.y(), robot_name)
                if task:
                    painter.drawText(position.x() + 25, position.y() + 15, task)

        for connection in self.connections:
            start, end, conn_type = connection
            self.draw_arrow(painter, start, end)
            self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end):
        line = QLineF(start, end)
        painter.drawLine(line)
        # Calculate arrow head
        arrow_size = 10
        angle = np.arctan2(-(end.y() - start.y()), end.x() - start.x())
        p1 = end + QPointF(np.cos(angle + np.pi / 3) * arrow_size,
                           np.sin(angle + np.pi / 3) * arrow_size)
        p2 = end + QPointF(np.cos(angle - np.pi / 3) * arrow_size,
                           np.sin(angle - np.pi / 3) * arrow_size)
        arrow_head = QPolygonF([end, p1, p2])
        painter.setBrush(Qt.black)
        painter.drawPolygon(arrow_head)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name and self.popup_task and self.popup_desc:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = event.pos()
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:  # Only add connection if popup is saved
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        # Update the behavior tree
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = event.pos()
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.edit_shape(clicked_shape)
            else:
                super(DropArea, self).mousePressEvent(event)

    def mouseDoubleClickEvent(self, event):
        clicked_pos = event.pos()
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        print(f"Double clicked at position: {clicked_pos}, detected shape: {clicked_shape}")  # Debug print
        if clicked_shape:
            self.edit_shape(clicked_shape)
        else:
            super(DropArea, self).mouseDoubleClickEvent(event)

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None




class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add drag-and-drop shapes to the sidebar
        self.circleLabel = ShapeLabel("Circle", self)
        self.squareLabel = ShapeLabel("Square", self)
        self.eraserLabel = EraserLabel(self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleLabel)
        self.shapeLayout.addWidget(self.squareLabel)
        self.shapeLayout.addWidget(self.eraserLabel)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.shapeContainer)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Add a drop area to the center
        self.dropArea = DropArea(self)
        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def closeEvent(self, event):
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    sys.exit(app.exec_())


In [ ]:
#this code has drag functionality of shapes to change posiions 
import sys
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QFrame, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QMouseEvent
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect
import subprocess
import time
import cv2
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class ShapeLabel(QLabel):
    def __init__(self, shape, parent=None):
        super(ShapeLabel, self).__init__(parent)
        self.shape = shape  # Circle or Square
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setText(shape)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class EraserLabel(QLabel):
    def __init__(self, parent=None):
        super(EraserLabel, self).__init__(parent)
        self.setText("Eraser")
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText("Eraser")

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None

class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = event.pos()
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = event.pos()
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = event.pos()
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos

                mimeData = QMimeData()
                mimeData.setText(shape)

                drag = QDrag(self)
                drag.setMimeData(mimeData)

                pixmap = QPixmap(50, 50)
                pixmap.fill(Qt.transparent)
                painter = QPainter(pixmap)
                painter.setPen(QPen(Qt.black, 2, Qt.SolidLine))
                if shape == "Circle":
                    painter.drawEllipse(10, 10, 30, 30)
                elif shape == "Square":
                    painter.drawRect(10, 10, 30, 30)
                painter.end()

                drag.setPixmap(pixmap)
                drag.setHotSpot(event.pos() - position)

                drag.exec_(Qt.MoveAction)
            else:
                self.double_click_timer = self.startTimer(200)

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = event.pos()
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(position.x() + 25, position.y(), robot_name)
                if task:
                    painter.drawText(position.x() + 25, position.y() + 15, task)

        for connection in self.connections:
            start, end, conn_type = connection
            self.draw_arrow(painter, start, end)
            self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end):
        line = QLineF(start, end)
        painter.drawLine(line)
        arrow_size = 10
        angle = np.arctan2(-(end.y() - start.y()), end.x() - start.x())
        p1 = end + QPointF(np.cos(angle + np.pi / 3) * arrow_size,
                           np.sin(angle + np.pi / 3) * arrow_size)
        p2 = end + QPointF(np.cos(angle - np.pi / 3) * arrow_size,
                           np.sin(angle - np.pi / 3) * arrow_size)
        arrow_head = QPolygonF([end, p1, p2])
        painter.setBrush(Qt.black)
        painter.drawPolygon(arrow_head)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add drag-and-drop shapes to the sidebar
        self.circleLabel = ShapeLabel("Circle", self)
        self.squareLabel = ShapeLabel("Square", self)
        self.eraserLabel = EraserLabel(self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleLabel)
        self.shapeLayout.addWidget(self.squareLabel)
        self.shapeLayout.addWidget(self.eraserLabel)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.shapeContainer)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Add a drop area to the center
        self.dropArea = DropArea(self)
        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def closeEvent(self, event):
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    sys.exit(app.exec_())


In [ ]:
# zoom in functionality 
import sys
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QFrame, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QMouseEvent
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint  # Add QPoint here
import subprocess
import time
import cv2
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class ShapeLabel(QLabel):
    def __init__(self, shape, parent=None):
        super(ShapeLabel, self).__init__(parent)
        self.shape = shape  # Circle or Square
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setText(shape)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)
            
            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class EraserLabel(QLabel):
    def __init__(self, parent=None):
        super(EraserLabel, self).__init__(parent)
        self.setText("Eraser")
        self.setFixedSize(50, 50)
        self.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.setLineWidth(2)
        self.setAutoFillBackground(True)
        self.setAlignment(Qt.AlignCenter)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText("Eraser")

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)

class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None

class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(position.x() + 25, position.y(), robot_name)
                if task:
                    painter.drawText(position.x() + 25, position.y() + 15, task)

        for connection in self.connections:
            start, end, conn_type = connection
            self.draw_arrow(painter, start, end)
            self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end):
        line = QLineF(start, end)
        painter.drawLine(line)
        arrow_size = 10
        angle = np.arctan2(-(end.y() - start.y()), end.x() - start.x())
        p1 = end + QPointF(np.cos(angle + np.pi / 3) * arrow_size,
                           np.sin(angle + np.pi / 3) * arrow_size)
        p2 = end + QPointF(np.cos(angle - np.pi / 3) * arrow_size,
                           np.sin(angle - np.pi / 3) * arrow_size)
        arrow_head = QPolygonF([end, p1, p2])
        painter.setBrush(Qt.black)
        painter.drawPolygon(arrow_head)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add drag-and-drop shapes to the sidebar
        self.circleLabel = ShapeLabel("Circle", self)
        self.squareLabel = ShapeLabel("Square", self)
        self.eraserLabel = EraserLabel(self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleLabel)
        self.shapeLayout.addWidget(self.squareLabel)
        self.shapeLayout.addWidget(self.eraserLabel)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.shapeContainer)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Add a drop area to the center
        self.dropArea = DropArea(self)
        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def closeEvent(self, event):
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    sys.exit(app.exec_())


In [ ]:
#BETTER AESTHETICS UI 
import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QFrame, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import time
import cv2
from main_gui import Ui_MainWindow  # Ensure this import matches the file name


class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogNoButton))
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(position.x() + 25, position.y(), robot_name)
                if task:
                    painter.drawText(position.x() + 25, position.y() + 15, task)

        for connection in self.connections:
            start, end, conn_type = connection
            self.draw_arrow(painter, start, end)
            self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end):
        line = QLineF(start, end)
        painter.drawLine(line)
        arrow_size = 10
        angle = np.arctan2(-(end.y() - start.y()), end.x() - start.x())
        p1 = end + QPointF(np.cos(angle + np.pi / 3) * arrow_size,
                           np.sin(angle + np.pi / 3) * arrow_size)
        p2 = end + QPointF(np.cos(angle - np.pi / 3) * arrow_size,
                           np.sin(angle - np.pi / 3) * arrow_size)
        arrow_head = QPolygonF([end, p1, p2])
        painter.setBrush(Qt.black)
        painter.drawPolygon(arrow_head)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)  # Add this line

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)  # Add DropArea to the layout

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree from the drop area
        behavior_tree = self.dropArea.behavior_tree

        # Convert behavior tree to a dictionary
        behavior_dict = {}
        for shape_id, details in behavior_tree.items():
            behavior_dict[details['name']] = {
                'task': details['task'],
                'additional_info': details['desc'],
                'child': [child['id'] for child in details['children']],
                'child_condition': [child['type'] for child in details['children']]
            }

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file:
            json.dump(behavior_dict, json_file, indent=4)
        print("Behavior tree saved to behavior_tree.json")

    def closeEvent(self, event):
        event.accept()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    sys.exit(app.exec_())


In [ ]:
#CHECKPOINT -1 
import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import time
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        



    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(position.x() + 25, position.y(), robot_name)
                if task:
                    painter.drawText(position.x() + 25, position.y() + 15, task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()


    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = start + direction * 20  # 20 is the radius for the circle
        corrected_end = end - direction * 20  # 20 is the radius for the circle

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)


    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    

    

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)  # Add this line

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)  # Add DropArea to the layout

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree from the drop area
        behavior_tree = self.dropArea.behavior_tree

        # Convert behavior tree to a dictionary
        behavior_dict = {}
        for shape_id, details in behavior_tree.items():
            behavior_dict[details['name']] = {
                'task': details['task'],
                'additional_info': details['desc'],
                'child': [child['id'] for child in details['children']],
                'child_condition': [child['type'] for child in details['children']]
            }

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file:
            json.dump(behavior_dict, json_file, indent=4)
        print("Behavior tree saved to behavior_tree.json")

    def closeEvent(self, event):
        event.accept()


if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())
 

In [ ]:
#partial history tab contructed 
import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle,QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import time
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class HistoryWidget(QWidget):
    def __init__(self, parent=None):
        super(HistoryWidget, self).__init__(parent)
        self.setWindowTitle("History Behavior Tree")
        self.setFixedSize(300, 400)
        self.layout = QVBoxLayout(self)

        self.history_list = QListWidget()
        self.history_list.setSelectionMode(QAbstractItemView.SingleSelection)
        self.history_list.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.layout.addWidget(self.history_list)

    def add_item(self, name, pixmap):
        item = QListWidgetItem(name)
        item.setData(Qt.UserRole, pixmap)

        # Create a label to show the pixmap
        label = QLabel()
        label.setPixmap(pixmap)
        label.setScaledContents(True)
        label.setFixedSize(280, 280)  # Adjust as needed

        # Add the label to the list item
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(label)
        layout.addStretch()

        item.setSizeHint(widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, widget)



class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        



    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
        return None

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(position.x() + 25, position.y(), robot_name)
                if task:
                    painter.drawText(position.x() + 25, position.y() + 15, task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()


    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = start + direction * 20  # 20 is the radius for the circle
        corrected_end = end - direction * 20  # 20 is the radius for the circle

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)


    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    

    

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(['roscore'])
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("roscore is already running")

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree from the drop area
        behavior_tree = self.dropArea.behavior_tree

        # Check if there's anything to save
        if not behavior_tree:
            print("No behavior tree to save")
            return

        # Convert behavior tree to a dictionary
        behavior_dict = {}
        for shape_id, details in behavior_tree.items():
            behavior_dict[details['name']] = {
                'task': details['task'],
                'additional_info': details['desc'],
                'child': [child['id'] for child in details['children']],
                'child_condition': [child['type'] for child in details['children']]
            }

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file:
            json.dump(behavior_dict, json_file, indent=4)
        print("Behavior tree saved to behavior_tree.json")

        # Create a pixmap of the current drop area
        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)

        # Add to history
        self.historyWidget.add_item("Behavior Tree", pixmap)

    def closeEvent(self, event):
        event.accept()


if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())

In [ ]:
#save as compound works
import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle, QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox, QMenu, QAction, QFileDialog)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import shlex
import time
import os
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class HistoryWidget(QWidget):
    def __init__(self, parent=None):
        super(HistoryWidget, self).__init__(parent)
        self.parent_widget = parent  # Store the reference to the parent widget
        self.setWindowTitle("History Behavior Tree")
        self.setFixedSize(300, 400)
        self.layout = QVBoxLayout(self)

        self.history_list = QListWidget()
        self.history_list.setSelectionMode(QAbstractItemView.SingleSelection)
        self.history_list.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.layout.addWidget(self.history_list)

        # Add a layout for the top buttons
        self.top_layout = QHBoxLayout()
        self.layout.addLayout(self.top_layout)

        # Add the three-line button
        self.menu_button = QPushButton("≡", self)
        self.menu_button.setFixedSize(30, 30)
        self.menu_button.clicked.connect(self.show_menu)
        self.top_layout.addWidget(self.menu_button)

        # Add the Select button
        self.select_button = QPushButton("Select", self)
        self.select_button.setFixedSize(70, 30)
        self.top_layout.addWidget(self.select_button)

        self.top_layout.addStretch()

    def show_menu(self):
        menu = QMenu(self)

        save_modify_action = QAction("Save and Modify", self)
        save_compound_action = QAction("Save as Compound", self)
        
        save_modify_action.triggered.connect(self.parent_widget.save_and_modify)
        save_compound_action.triggered.connect(self.parent_widget.save_as_compound)
        
        menu.addAction(save_modify_action)
        menu.addAction(save_compound_action)

        menu.exec_(self.menu_button.mapToGlobal(self.menu_button.rect().bottomLeft()))

    def add_item(self, name, pixmap):
        item = QListWidgetItem(name)
        item.setData(Qt.UserRole, pixmap)

        # Create a label to show the pixmap
        label = QLabel()
        label.setPixmap(pixmap)
        label.setScaledContents(True)
        label.setFixedSize(280, 280)  # Adjust as needed

        # Add the label to the list item
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(label)
        layout.addStretch()

        item.setSizeHint(widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, widget)

class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Diamond":
            # Create a custom pixmap for a diamond icon
            pixmap = QPixmap(32, 32)
            pixmap.fill(Qt.transparent)
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))
            painter.setBrush(Qt.gray)
            points = [QPointF(16, 4), QPointF(28, 16), QPointF(16, 28), QPointF(4, 16)]
            painter.drawPolygon(QPolygonF(points))
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree from the drop area
        behavior_tree = self.dropArea.behavior_tree

        # Check if there's anything to save
        if not behavior_tree:
            print("No behavior tree to save")
            return

        # Convert behavior tree to a dictionary
        behavior_dict = {}
        for shape_id, details in behavior_tree.items():
            behavior_dict[details['name']] = {
                'task': details['task'],
                'additional_info': details['desc'],
                'child': [child['id'] for child in details['children']],
                'child_condition': [child['type'] for child in details['children']]
            }

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file:
            json.dump(behavior_dict, json_file, indent=4)
        print("Behavior tree saved to behavior_tree.json")

        # Create a pixmap of the current drop area
        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)

        # Add to history
        self.historyWidget.add_item("Behavior Tree", pixmap)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {}
            for shape_id, details in behavior_tree.items():
                behavior_dict[shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")
                self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # First, load all shapes
        for shape_id, details in behavior_dict.items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Then, load all connections
        for shape_id, details in self.dropArea.behavior_tree.items():
            for child in details['children']:
                start_pos = QPointF(details['position'][0], details['position'][1])
                if child['id'] in self.dropArea.behavior_tree:
                    end_details = self.dropArea.behavior_tree[child['id']]
                    end_pos = QPointF(end_details['position'][0], end_details['position'][1])
                    print(f"Connecting {start_pos} to {end_pos} with type {child['type']}")
                    self.dropArea.connections.append((start_pos, end_pos, child['type']))
                else:
                    print(f"Error: Child id {child['id']} not found in behavior_tree")

        self.dropArea.update()
        print("Finished loading behavior tree")

if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())


In [ ]:
#save and modify + save as compound
import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle, QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox, QMenu, QAction, QFileDialog)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import shlex
import time
import os
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class HistoryWidget(QWidget):
    def __init__(self, parent=None):
        super(HistoryWidget, self).__init__(parent)
        self.parent_widget = parent  # Store the reference to the parent widget
        self.setWindowTitle("History Behavior Tree")
        self.setFixedSize(300, 400)
        self.layout = QVBoxLayout(self)

        self.history_list = QListWidget()
        self.history_list.setSelectionMode(QAbstractItemView.SingleSelection)
        self.history_list.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.layout.addWidget(self.history_list)

        # Add a layout for the top buttons
        self.top_layout = QHBoxLayout()
        self.layout.addLayout(self.top_layout)

        # Add the three-line button
        self.menu_button = QPushButton("≡", self)
        self.menu_button.setFixedSize(30, 30)
        self.menu_button.clicked.connect(self.show_menu)
        self.top_layout.addWidget(self.menu_button)

        # Add the Select button
        self.select_button = QPushButton("Select", self)
        self.select_button.setFixedSize(70, 30)
        self.top_layout.addWidget(self.select_button)

        self.top_layout.addStretch()

    def show_menu(self):
        menu = QMenu(self)

        save_modify_action = QAction("Save and Modify", self)
        save_compound_action = QAction("Save as Compound", self)
        
        save_modify_action.triggered.connect(self.parent_widget.save_and_modify)
        save_compound_action.triggered.connect(self.parent_widget.save_as_compound)
        
        menu.addAction(save_modify_action)
        menu.addAction(save_compound_action)

        menu.exec_(self.menu_button.mapToGlobal(self.menu_button.rect().bottomLeft()))

    def add_item(self, name, pixmap):
        item = QListWidgetItem(name)
        item.setData(Qt.UserRole, pixmap)

        # Create a label to show the pixmap
        label = QLabel()
        label.setPixmap(pixmap)
        label.setScaledContents(True)
        label.setFixedSize(280, 280)  # Adjust as needed

        # Add the label to the list item
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(label)
        layout.addStretch()

        item.setSizeHint(widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, widget)

class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Diamond":
            # Create a custom pixmap for a diamond icon
            pixmap = QPixmap(32, 32)
            pixmap.fill(Qt.transparent)
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))
            painter.setBrush(Qt.gray)
            points = [QPointF(16, 4), QPointF(28, 16), QPointF(16, 28), QPointF(4, 16)]
            painter.drawPolygon(QPolygonF(points))
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree from the drop area
        behavior_tree = self.dropArea.behavior_tree

        # Check if there's anything to save
        if not behavior_tree:
            print("No behavior tree to save")
            return

        # Convert behavior tree to a dictionary
        behavior_dict = {}
        for shape_id, details in behavior_tree.items():
            behavior_dict[shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file:
            json.dump(behavior_dict, json_file, indent=4)
        print("Behavior tree saved to behavior_tree.json")

        # Create a pixmap of the current drop area
        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)

        # Add to history
        self.historyWidget.add_item("Behavior Tree", pixmap)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {}
            for shape_id, details in behavior_tree.items():
                behavior_dict[shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")
                self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # First, load all shapes
        for shape_id, details in behavior_dict.items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Then, load all connections
        for shape_id, details in self.dropArea.behavior_tree.items():
            for child in details['children']:
                start_pos = QPointF(details['position'][0], details['position'][1])
                if child['id'] in self.dropArea.behavior_tree:
                    end_details = self.dropArea.behavior_tree[child['id']]
                    end_pos = QPointF(end_details['position'][0], end_details['position'][1])
                    print(f"Connecting {start_pos} to {end_pos} with type {child['type']}")
                    self.dropArea.connections.append((start_pos, end_pos, child['type']))
                else:
                    print(f"Error: Child id {child['id']} not found in behavior_tree")

        self.dropArea.update()
        print("Finished loading behavior tree")

if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())


In [ ]:
#has svae behavior tree functionalities 
import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle, QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox, QMenu, QAction, QFileDialog)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import shlex
import time
import os
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

class HistoryWidget(QWidget):
    def __init__(self, parent=None):
        super(HistoryWidget, self).__init__(parent)
        self.parent_widget = parent  # Store the reference to the parent widget
        self.setWindowTitle("History Behavior Tree")
        self.setFixedSize(300, 400)
        self.layout = QVBoxLayout(self)

        self.history_list = QListWidget()
        self.history_list.setSelectionMode(QAbstractItemView.SingleSelection)
        self.history_list.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.layout.addWidget(self.history_list)

        # Add a layout for the top buttons
        self.top_layout = QHBoxLayout()
        self.layout.addLayout(self.top_layout)

        # Add the three-line button
        self.menu_button = QPushButton("≡", self)
        self.menu_button.setFixedSize(30, 30)
        self.menu_button.clicked.connect(self.show_menu)
        self.top_layout.addWidget(self.menu_button)

        # Add the Select button
        self.select_button = QPushButton("Select", self)
        self.select_button.setFixedSize(70, 30)
        self.select_button.clicked.connect(self.select_behavior_tree)  # Connect the Select button
        self.top_layout.addWidget(self.select_button)

        self.top_layout.addStretch()

        self.saved_behavior_trees = []  # List to store saved behavior trees

    def show_menu(self):
        menu = QMenu(self)

        save_modify_action = QAction("Save and Modify", self)
        save_compound_action = QAction("Save as Compound", self)
        
        save_modify_action.triggered.connect(self.parent_widget.save_and_modify)
        save_compound_action.triggered.connect(self.parent_widget.save_as_compound)
        
        menu.addAction(save_modify_action)
        menu.addAction(save_compound_action)

        menu.exec_(self.menu_button.mapToGlobal(self.menu_button.rect().bottomLeft()))

    def add_item(self, name, pixmap, behavior_tree):
        item = QListWidgetItem(name)
        item.setData(Qt.UserRole, pixmap)
        item.setData(Qt.UserRole + 1, behavior_tree)  # Store the behavior tree in the item

        # Create a label to show the pixmap
        label = QLabel()
        label.setPixmap(pixmap)
        label.setScaledContents(True)
        label.setFixedSize(280, 280)  # Adjust as needed

        # Add the label to the list item
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(label)
        layout.addStretch()

        item.setSizeHint(widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, widget)

    def select_behavior_tree(self):
        selected_items = self.history_list.selectedItems()
        if not selected_items:
            QMessageBox.warning(self, "Selection Error", "Please select a behavior tree from the history.")
            return

        selected_item = selected_items[0]
        behavior_tree = selected_item.data(Qt.UserRole + 1)  # Retrieve the stored behavior tree
        self.parent_widget.load_behavior_tree(behavior_tree)  # Call the parent's load method


class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Diamond":
            # Create a custom pixmap for a diamond icon
            pixmap = QPixmap(32, 32)
            pixmap.fill(Qt.transparent)
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))
            painter.setBrush(Qt.gray)
            points = [QPointF(16, 4), QPointF(28, 16), QPointF(16, 28), QPointF(4, 16)]
            painter.drawPolygon(QPolygonF(points))
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree from the drop area
        behavior_tree = self.dropArea.behavior_tree

        # Check if there's anything to save
        if not behavior_tree:
            print("No behavior tree to save")
            return

        # Convert behavior tree to a dictionary
        behavior_dict = {}
        for shape_id, details in behavior_tree.items():
            behavior_dict[shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file, open('behavior_tree_history.json', 'a') as history_file:
            json.dump(behavior_dict, json_file, indent=4)
            history_file.write(json.dumps(behavior_dict) + "\n")  # Save to history file
        print("Behavior tree saved to behavior_tree.json and appended to history")

        # Create a pixmap of the current drop area
        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)

        # Add to history
        self.historyWidget.add_item("Behavior Tree", pixmap, behavior_dict)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {}
            for shape_id, details in behavior_tree.items():
                behavior_dict[shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")
                self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # First, load all shapes
        for shape_id, details in behavior_dict.items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Then, load all connections
        for shape_id, details in self.dropArea.behavior_tree.items():
            for child in details['children']:
                start_pos = QPointF(details['position'][0], details['position'][1])
                if child['id'] in self.dropArea.behavior_tree:
                    end_details = self.dropArea.behavior_tree[child['id']]
                    end_pos = QPointF(end_details['position'][0], end_details['position'][1])
                    print(f"Connecting {start_pos} to {end_pos} with type {child['type']}")
                    self.dropArea.connections.append((start_pos, end_pos, child['type']))
                else:
                    print(f"Error: Child id {child['id']} not found in behavior_tree")

        self.dropArea.update()
        print("Finished loading behavior tree")

if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())


In [ ]:
#combines small trees wo overlPPING 
import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle, QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox, QMenu, QAction, QFileDialog)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import shlex
import time
import os
from main_gui import Ui_MainWindow  # Ensure this import matches the file name
import math


class HistoryWidget(QWidget):
    def __init__(self, parent=None):
        super(HistoryWidget, self).__init__(parent)
        self.parent_widget = parent  # Store the reference to the parent widget
        self.setWindowTitle("History Behavior Tree")
        self.setFixedSize(300, 400)
        self.layout = QVBoxLayout(self)

        self.history_list = QListWidget()
        self.history_list.setSelectionMode(QAbstractItemView.MultiSelection)  # Enable multiple selection
        self.history_list.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.layout.addWidget(self.history_list)

        # Add a layout for the top buttons
        self.top_layout = QHBoxLayout()
        self.layout.addLayout(self.top_layout)

        # Add the three-line button
        self.menu_button = QPushButton("≡", self)
        self.menu_button.setFixedSize(30, 30)
        self.menu_button.clicked.connect(self.show_menu)
        self.top_layout.addWidget(self.menu_button)

        # Add the Select button
        self.select_button = QPushButton("Select", self)
        self.select_button.setFixedSize(70, 30)
        self.select_button.clicked.connect(self.select_behavior_tree)  # Connect the Select button
        self.top_layout.addWidget(self.select_button)

        self.top_layout.addStretch()

        self.saved_behavior_trees = []  # List to store saved behavior trees

    def show_menu(self):
        menu = QMenu(self)

        save_modify_action = QAction("Save and Modify", self)
        save_compound_action = QAction("Save as Compound", self)
        
        save_modify_action.triggered.connect(self.parent_widget.save_and_modify)
        save_compound_action.triggered.connect(self.parent_widget.save_as_compound)
        
        menu.addAction(save_modify_action)
        menu.addAction(save_compound_action)

        menu.exec_(self.menu_button.mapToGlobal(self.menu_button.rect().bottomLeft()))

    def add_item(self, name, pixmap, behavior_tree):
        item = QListWidgetItem(name)
        item.setData(Qt.UserRole, pixmap)
        item.setData(Qt.UserRole + 1, behavior_tree)  # Store the behavior tree in the item

        # Create a label to show the pixmap
        label = QLabel()
        label.setPixmap(pixmap)
        label.setScaledContents(True)
        label.setFixedSize(280, 280)  # Adjust as needed

        # Add the label to the list item
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(label)
        layout.addStretch()

        item.setSizeHint(widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, widget)

    def select_behavior_tree(self):
        selected_items = self.history_list.selectedItems()
        if not selected_items:
            QMessageBox.warning(self, "Selection Error", "Please select one or more behavior trees from the history.")
            return

        combined_behavior_tree = {}
        combined_connections = []
        offset_y = 0  # Initial offset

        for selected_item in selected_items:
            behavior_tree = selected_item.data(Qt.UserRole + 1)  # Retrieve the stored behavior tree
            combined_behavior_tree, combined_connections = self.combine_trees(combined_behavior_tree, combined_connections, behavior_tree, offset_y)
            offset_y += 200  # Increase offset to stack trees vertically

        print("Combined Behavior Tree:", combined_behavior_tree)
        print("Combined Connections:", combined_connections)
        self.parent_widget.load_behavior_tree(combined_behavior_tree, combined_connections)  # Call the parent's load method

    def combine_trees(self, tree1, connections1, tree2, offset_y):
        combined_tree = tree1.copy()
        combined_connections = connections1.copy()

        # Create a mapping from old IDs to new IDs
        id_mapping = {}

        # Add shapes to the combined tree
        for shape_id, details in tree2.items():
            new_shape_id = f"{shape_id}_{len(combined_tree)}"  # Unique ID
            new_position = QPointF(details['position'][0], details['position'][1] + offset_y)
            combined_tree[new_shape_id] = {
                'type': details['type'],
                'position': (new_position.x(), new_position.y()),
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }
            id_mapping[shape_id] = new_shape_id
            print(f"Added shape {new_shape_id} at position {new_position}")

        # Add connections to the combined tree
        for shape_id, details in tree2.items():
            new_shape_id = id_mapping[shape_id]
            for child in details['children']:
                old_child_id = child['id']
                if old_child_id in id_mapping:
                    new_child_id = id_mapping[old_child_id]
                    start_position = QPointF(combined_tree[new_shape_id]['position'][0], combined_tree[new_shape_id]['position'][1])
                    end_position = QPointF(combined_tree[new_child_id]['position'][0], combined_tree[new_child_id]['position'][1])
                    combined_connections.append((start_position, end_position, child['type']))
                    print(f"Connected {start_position} to {end_position} with type {child['type']}")

        return combined_tree, combined_connections


class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Diamond":
            # Create a custom pixmap for a diamond icon
            pixmap = QPixmap(32, 32)
            pixmap.fill(Qt.transparent)
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))
            painter.setBrush(Qt.gray)
            points = [QPointF(16, 4), QPointF(28, 16), QPointF(16, 28), QPointF(4, 16)]
            painter.drawPolygon(QPolygonF(points))
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(int(position.x() - 20), int(position.y() - 20), 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree from the drop area
        behavior_tree = self.dropArea.behavior_tree

        # Check if there's anything to save
        if not behavior_tree:
            print("No behavior tree to save")
            return

        # Convert behavior tree to a dictionary
        behavior_dict = {}
        for shape_id, details in behavior_tree.items():
            behavior_dict[shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file, open('behavior_tree_history.json', 'a') as history_file:
            json.dump(behavior_dict, json_file, indent=4)
            history_file.write(json.dumps(behavior_dict) + "\n")  # Save to history file
        print("Behavior tree saved to behavior_tree.json and appended to history")

        # Create a pixmap of the current drop area
        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)

        # Add to history
        self.historyWidget.add_item("Behavior Tree", pixmap, behavior_dict)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {}
            for shape_id, details in behavior_tree.items():
                behavior_dict[shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")
                self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict, connections=[]):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # First, load all shapes
        for shape_id, details in behavior_dict.items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Then, load all connections
        if not connections:
            for shape_id, details in self.dropArea.behavior_tree.items():
                for child in details['children']:
                    start_pos = QPointF(details['position'][0], details['position'][1])
                    if child['id'] in self.dropArea.behavior_tree:
                        end_details = self.dropArea.behavior_tree[child['id']]
                        end_pos = QPointF(end_details['position'][0], end_details['position'][1])
                        print(f"Connecting {start_pos} to {end_pos} with type {child['type']}")
                        self.dropArea.connections.append((start_pos, end_pos, child['type']))
                    else:
                        print(f"Error: Child id {child['id']} not found in behavior_tree")
        else:
            self.dropArea.connections = connections

        self.dropArea.update()
        print("Finished loading behavior tree")

    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree from the drop area
        behavior_tree = self.dropArea.behavior_tree

        # Check if there's anything to save
        if not behavior_tree:
            print("No behavior tree to save")
            return

        # Convert behavior tree to a dictionary
        behavior_dict = {}
        for shape_id, details in behavior_tree.items():
            behavior_dict[shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file, open('behavior_tree_history.json', 'a') as history_file:
            json.dump(behavior_dict, json_file, indent=4)
            history_file.write(json.dumps(behavior_dict) + "\n")  # Save to history file
        print("Behavior tree saved to behavior_tree.json and appended to history")

        # Create a pixmap of the current drop area
        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)

        # Add to history
        self.historyWidget.add_item("Behavior Tree", pixmap, behavior_dict)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {}
            for shape_id, details in behavior_tree.items():
                behavior_dict[shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")
                self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict, connections=[]):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # First, load all shapes
        for shape_id, details in behavior_dict.items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Then, load all connections
        if not connections:
            for shape_id, details in self.dropArea.behavior_tree.items():
                for child in details['children']:
                    start_pos = QPointF(details['position'][0], details['position'][1])
                    if child['id'] in self.dropArea.behavior_tree:
                        end_details = self.dropArea.behavior_tree[child['id']]
                        end_pos = QPointF(end_details['position'][0], end_details['position'][1])
                        print(f"Connecting {start_pos} to {end_pos} with type {child['type']}")
                        self.dropArea.connections.append((start_pos, end_pos, child['type']))
                    else:
                        print(f"Error: Child id {child['id']} not found in behavior_tree")
        else:
            self.dropArea.connections = connections

        self.dropArea.update()
        print("Finished loading behavior tree")


if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())

    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree from the drop area
        behavior_tree = self.dropArea.behavior_tree

        # Check if there's anything to save
        if not behavior_tree:
            print("No behavior tree to save")
            return

        # Convert behavior tree to a dictionary
        behavior_dict = {}
        for shape_id, details in behavior_tree.items():
            behavior_dict[shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file, open('behavior_tree_history.json', 'a') as history_file:
            json.dump(behavior_dict, json_file, indent=4)
            history_file.write(json.dumps(behavior_dict) + "\n")  # Save to history file
        print("Behavior tree saved to behavior_tree.json and appended to history")

        # Create a pixmap of the current drop area
        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)

        # Add to history
        self.historyWidget.add_item("Behavior Tree", pixmap, behavior_dict)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {}
            for shape_id, details in behavior_tree.items():
                behavior_dict[shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")
                self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # First, load all shapes
        for shape_id, details in behavior_dict.items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Then, load all connections
        for shape_id, details in self.dropArea.behavior_tree.items():
            for child in details['children']:
                start_pos = QPointF(details['position'][0], details['position'][1])
                if child['id'] in self.dropArea.behavior_tree:
                    end_details = self.dropArea.behavior_tree[child['id']]
                    end_pos = QPointF(end_details['position'][0], end_details['position'][1])
                    print(f"Connecting {start_pos} to {end_pos} with type {child['type']}")
                    self.dropArea.connections.append((start_pos, end_pos, child['type']))
                else:
                    print(f"Error: Child id {child['id']} not found in behavior_tree")

        self.dropArea.update()
        print("Finished loading behavior tree")

if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())


In [ ]:
#COMBINES EVERYTHING PROPERLY FROM HISTORIY TAB 
#combines small trees wo overlPPING 
import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle, QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox, QMenu, QAction, QFileDialog)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import shlex
import time
import os
from main_gui import Ui_MainWindow  # Ensure this import matches the file name
import math


class HistoryWidget(QWidget):
    def __init__(self, parent=None):
        super(HistoryWidget, self).__init__(parent)
        self.parent_widget = parent  # Store the reference to the parent widget
        self.setWindowTitle("History Behavior Tree")
        self.setFixedSize(300, 400)
        self.layout = QVBoxLayout(self)

        self.history_list = QListWidget()
        self.history_list.setSelectionMode(QAbstractItemView.MultiSelection)  # Enable multiple selection
        self.history_list.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.layout.addWidget(self.history_list)

        # Add a layout for the top buttons
        self.top_layout = QHBoxLayout()
        self.layout.addLayout(self.top_layout)

        # Add the three-line button
        self.menu_button = QPushButton("≡", self)
        self.menu_button.setFixedSize(30, 30)
        self.menu_button.clicked.connect(self.show_menu)
        self.top_layout.addWidget(self.menu_button)

        # Add the Select button
        self.select_button = QPushButton("Select", self)
        self.select_button.setFixedSize(70, 30)
        self.select_button.clicked.connect(self.select_behavior_tree)  # Connect the Select button
        self.top_layout.addWidget(self.select_button)

        self.top_layout.addStretch()

        self.saved_behavior_trees = []  # List to store saved behavior trees

    def show_menu(self):
        menu = QMenu(self)

        save_modify_action = QAction("Save and Modify", self)
        save_compound_action = QAction("Save as Compound", self)
        
        save_modify_action.triggered.connect(self.parent_widget.save_and_modify)
        save_compound_action.triggered.connect(self.parent_widget.save_as_compound)
        
        menu.addAction(save_modify_action)
        menu.addAction(save_compound_action)

        menu.exec_(self.menu_button.mapToGlobal(self.menu_button.rect().bottomLeft()))

    def add_item(self, name, pixmap, behavior_tree):
        item = QListWidgetItem(name)
        item.setData(Qt.UserRole, pixmap)
        item.setData(Qt.UserRole + 1, behavior_tree)  # Store the behavior tree in the item

        # Create a label to show the pixmap
        label = QLabel()
        label.setPixmap(pixmap)
        label.setScaledContents(True)
        label.setFixedSize(280, 280)  # Adjust as needed

        # Add the label to the list item
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(label)
        layout.addStretch()

        item.setSizeHint(widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, widget)

    def select_behavior_tree(self):
        selected_items = self.history_list.selectedItems()
        if not selected_items:
            QMessageBox.warning(self, "Selection Error", "Please select one or more behavior trees from the history.")
            return

        combined_behavior_tree = {}
        combined_connections = []
        offset_y = 0  # Initial offset

        for selected_item in selected_items:
            behavior_tree = selected_item.data(Qt.UserRole + 1)  # Retrieve the stored behavior tree
            combined_behavior_tree, combined_connections = self.combine_trees(combined_behavior_tree, combined_connections, behavior_tree, offset_y)
            offset_y += 200  # Increase offset to stack trees vertically

        print("Combined Behavior Tree:", combined_behavior_tree)
        print("Combined Connections:", combined_connections)
        self.parent_widget.load_behavior_tree(combined_behavior_tree, combined_connections)  # Call the parent's load method

    def combine_trees(self, tree1, connections1, tree2, offset_y):
        combined_tree = tree1.copy()
        combined_connections = connections1.copy()

        # Create a mapping from old IDs to new IDs
        id_mapping = {}

        # Add shapes to the combined tree
        for shape_id, details in tree2.items():
            new_shape_id = f"{shape_id}_{len(combined_tree)}"  # Unique ID
            new_position = QPointF(details['position'][0], details['position'][1] + offset_y)
            combined_tree[new_shape_id] = {
                'type': details['type'],
                'position': (new_position.x(), new_position.y()),
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }
            id_mapping[shape_id] = new_shape_id
            print(f"Added shape {new_shape_id} at position {new_position}")

        # Add connections to the combined tree
        for shape_id, details in tree2.items():
            new_shape_id = id_mapping[shape_id]
            for child in details['children']:
                old_child_id = child['id']
                if old_child_id in id_mapping:
                    new_child_id = id_mapping[old_child_id]
                    start_position = QPointF(combined_tree[new_shape_id]['position'][0], combined_tree[new_shape_id]['position'][1])
                    end_position = QPointF(combined_tree[new_child_id]['position'][0], combined_tree[new_child_id]['position'][1])
                    combined_connections.append((start_position, end_position, child['type']))
                    print(f"Connected {start_position} to {end_position} with type {child['type']}")

        return combined_tree, combined_connections



class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Diamond":
            # Create a custom pixmap for a diamond icon
            pixmap = QPixmap(32, 32)
            pixmap.fill(Qt.transparent)
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))
            painter.setBrush(Qt.gray)
            points = [QPointF(16, 4), QPointF(28, 16), QPointF(16, 28), QPointF(4, 16)]
            painter.drawPolygon(QPolygonF(points))
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(int(position.x() - 20), int(position.y() - 20), 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree and connections from the drop area
        behavior_tree = self.dropArea.behavior_tree
        connections = self.dropArea.connections

        # Check if there's anything to save
        if not behavior_tree:
            print("No behavior tree to save")
            return

        # Convert behavior tree and connections to dictionaries
        behavior_dict = {
            'shapes': {},
            'connections': []
        }
        for shape_id, details in behavior_tree.items():
            behavior_dict['shapes'][shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        for connection in connections:
            start, end, conn_type = connection
            behavior_dict['connections'].append({
                'start': (start.x(), start.y()),
                'end': (end.x(), end.y()),
                'type': conn_type
            })

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file, open('behavior_tree_history.json', 'a') as history_file:
            json.dump(behavior_dict, json_file, indent=4)
            history_file.write(json.dumps(behavior_dict) + "\n")  # Save to history file
        print("Behavior tree saved to behavior_tree.json and appended to history")

        # Create a pixmap of the current drop area
        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)

        # Add to history
        self.historyWidget.add_item("Behavior Tree", pixmap, behavior_dict)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {}
            for shape_id, details in behavior_tree.items():
                behavior_dict[shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")
                self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict, connections=[]):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # Load shapes
        for shape_id, details in behavior_dict['shapes'].items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Load connections
        if not connections:
            for connection in behavior_dict['connections']:
                start = QPointF(connection['start'][0], connection['start'][1])
                end = QPointF(connection['end'][0], connection['end'][1])
                conn_type = connection['type']
                print(f"Connecting {start} to {end} with type {conn_type}")
                self.dropArea.connections.append((start, end, conn_type))
        else:
            self.dropArea.connections = connections

        self.dropArea.update()
        print("Finished loading behavior tree")

    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree from the drop area
        behavior_tree = self.dropArea.behavior_tree

        # Check if there's anything to save
        if not behavior_tree:
            print("No behavior tree to save")
            return

        # Convert behavior tree to a dictionary
        behavior_dict = {}
        for shape_id, details in behavior_tree.items():
            behavior_dict[shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file, open('behavior_tree_history.json', 'a') as history_file:
            json.dump(behavior_dict, json_file, indent=4)
            history_file.write(json.dumps(behavior_dict) + "\n")  # Save to history file
        print("Behavior tree saved to behavior_tree.json and appended to history")

        # Create a pixmap of the current drop area
        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)

        # Add to history
        self.historyWidget.add_item("Behavior Tree", pixmap, behavior_dict)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {}
            for shape_id, details in behavior_tree.items():
                behavior_dict[shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")
                self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict, connections=[]):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # First, load all shapes
        for shape_id, details in behavior_dict.items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Then, load all connections
        if not connections:
            for shape_id, details in self.dropArea.behavior_tree.items():
                for child in details['children']:
                    start_pos = QPointF(details['position'][0], details['position'][1])
                    if child['id'] in self.dropArea.behavior_tree:
                        end_details = self.dropArea.behavior_tree[child['id']]
                        end_pos = QPointF(end_details['position'][0], end_details['position'][1])
                        print(f"Connecting {start_pos} to {end_pos} with type {child['type']}")
                        self.dropArea.connections.append((start_pos, end_pos, child['type']))
                    else:
                        print(f"Error: Child id {child['id']} not found in behavior_tree")
        else:
            self.dropArea.connections = connections

        self.dropArea.update()
        print("Finished loading behavior tree")


if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())

 

In [ ]:
#DARL THEME BUT PPOP UP DIALOG IS WEIRD 
import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle, QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox, QMenu, QAction, QFileDialog, QCheckBox)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import shlex
import time
import os
from main_gui import Ui_MainWindow  # Ensure this import matches the file name
import math


class HistoryWidget(QWidget):
    def __init__(self, parent=None):
        super(HistoryWidget, self).__init__(parent)
        self.parent_widget = parent  # Store the reference to the parent widget
        self.setWindowTitle("History Behavior Tree")
        self.setFixedSize(300, 400)
        self.layout = QVBoxLayout(self)

        self.history_list = QListWidget()
        self.history_list.setSelectionMode(QAbstractItemView.MultiSelection)  # Enable multiple selection
        self.history_list.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.layout.addWidget(self.history_list)

        # Add a layout for the top buttons
        self.top_layout = QHBoxLayout()
        self.layout.addLayout(self.top_layout)

        # Add the three-line button
        self.menu_button = QPushButton("≡", self)
        self.menu_button.setFixedSize(30, 30)
        self.menu_button.clicked.connect(self.show_menu)
        self.top_layout.addWidget(self.menu_button)

        # Add the Select button
        self.select_button = QPushButton("Select", self)
        self.select_button.setFixedSize(70, 30)
        self.select_button.clicked.connect(self.select_behavior_tree)  # Connect the Select button
        self.top_layout.addWidget(self.select_button)

        # Add the theme toggle checkbox
        self.themeToggleButton = QCheckBox("Dark Theme")
        self.themeToggleButton.stateChanged.connect(self.parent_widget.toggle_theme)
        self.top_layout.addWidget(self.themeToggleButton)

        self.top_layout.addStretch()

        self.saved_behavior_trees = []  # List to store saved behavior trees

    def show_menu(self):
        menu = QMenu(self)

        save_modify_action = QAction("Save and Modify", self)
        save_compound_action = QAction("Save as Compound", self)
        
        save_modify_action.triggered.connect(self.parent_widget.save_and_modify)
        save_compound_action.triggered.connect(self.parent_widget.save_as_compound)
        
        menu.addAction(save_modify_action)
        menu.addAction(save_compound_action)

        menu.exec_(self.menu_button.mapToGlobal(self.menu_button.rect().bottomLeft()))

    def add_item(self, name, pixmap, behavior_tree):
        item = QListWidgetItem(name)
        item.setData(Qt.UserRole, pixmap)
        item.setData(Qt.UserRole + 1, behavior_tree)  # Store the behavior tree in the item

        # Create a label to show the pixmap
        label = QLabel()
        label.setPixmap(pixmap)
        label.setScaledContents(True)
        label.setFixedSize(280, 280)  # Adjust as needed

        # Add the label to the list item
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(label)
        layout.addStretch()

        item.setSizeHint(widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, widget)

    def select_behavior_tree(self):
        selected_items = self.history_list.selectedItems()
        if not selected_items:
            QMessageBox.warning(self, "Selection Error", "Please select one or more behavior trees from the history.")
            return

        combined_behavior_tree = {}
        combined_connections = []
        offset_y = 0  # Initial offset

        for selected_item in selected_items:
            behavior_tree = selected_item.data(Qt.UserRole + 1)  # Retrieve the stored behavior tree
            combined_behavior_tree, combined_connections = self.combine_trees(combined_behavior_tree, combined_connections, behavior_tree, offset_y)
            offset_y += 200  # Increase offset to stack trees vertically

        print("Combined Behavior Tree:", combined_behavior_tree)
        print("Combined Connections:", combined_connections)
        self.parent_widget.load_behavior_tree(combined_behavior_tree, combined_connections)  # Call the parent's load method

    def combine_trees(self, tree1, connections1, tree2, offset_y):
        combined_tree = tree1.copy()
        combined_connections = connections1.copy()

        # Create a mapping from old IDs to new IDs
        id_mapping = {}

        # Add shapes to the combined tree
        for shape_id, details in tree2.items():
            new_shape_id = f"{shape_id}_{len(combined_tree)}"  # Unique ID
            new_position = QPointF(details['position'][0], details['position'][1] + offset_y)
            combined_tree[new_shape_id] = {
                'type': details['type'],
                'position': (new_position.x(), new_position.y()),
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }
            id_mapping[shape_id] = new_shape_id
            print(f"Added shape {new_shape_id} at position {new_position}")

        # Add connections to the combined tree
        for shape_id, details in tree2.items():
            new_shape_id = id_mapping[shape_id]
            for child in details['children']:
                old_child_id = child['id']
                if old_child_id in id_mapping:
                    new_child_id = id_mapping[old_child_id]
                    start_position = QPointF(combined_tree[new_shape_id]['position'][0], combined_tree[new_shape_id]['position'][1])
                    end_position = QPointF(combined_tree[new_child_id]['position'][0], combined_tree[new_child_id]['position'][1])
                    combined_connections.append((start_position, end_position, child['type']))
                    print(f"Connected {start_position} to {end_position} with type {child['type']}")

        return combined_tree, combined_connections



class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Diamond":
            # Create a custom pixmap for a diamond icon
            pixmap = QPixmap(32, 32)
            pixmap.fill(Qt.transparent)
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))
            painter.setBrush(Qt.gray)
            points = [QPointF(16, 4), QPointF(28, 16), QPointF(16, 28), QPointF(4, 16)]
            painter.drawPolygon(QPolygonF(points))
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(int(position.x() - 20), int(position.y() - 20), 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        # Initialize ROS core
        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

        # Apply the initial theme
        self.apply_light_theme()

    def apply_light_theme(self):
        self.load_qss("light.qss")

    def apply_dark_theme(self):
        self.load_qss("dark.qss")

    def load_qss(self, file_name):
        try:
            qss_file_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), file_name)
            with open(qss_file_path, "r") as f:
                self.setStyleSheet(f.read())
        except FileNotFoundError:
            print(f"QSS file '{file_name}' not found.")
        except Exception as e:
            print(f"Error loading QSS file '{file_name}': {e}")

    def toggle_theme(self, state):
        if state == Qt.Checked:
            self.apply_dark_theme()
        else:
            self.apply_light_theme()


    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree and connections from the drop area
        behavior_tree = self.dropArea.behavior_tree
        connections = self.dropArea.connections

        # Check if there's anything to save
        if not behavior_tree:
            print("No behavior tree to save")
            return

        # Convert behavior tree and connections to dictionaries
        behavior_dict = {
            'shapes': {},
            'connections': []
        }
        for shape_id, details in behavior_tree.items():
            behavior_dict['shapes'][shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        for connection in connections:
            start, end, conn_type = connection
            behavior_dict['connections'].append({
                'start': (start.x(), start.y()),
                'end': (end.x(), end.y()),
                'type': conn_type
            })

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file, open('behavior_tree_history.json', 'a') as history_file:
            json.dump(behavior_dict, json_file, indent=4)
            history_file.write(json.dumps(behavior_dict) + "\n")  # Save to history file
        print("Behavior tree saved to behavior_tree.json and appended to history")

        # Create a pixmap of the current drop area
        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)

        # Add to history
        self.historyWidget.add_item("Behavior Tree", pixmap, behavior_dict)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {}
            for shape_id, details in behavior_tree.items():
                behavior_dict[shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")
                self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict, connections=[]):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # Load shapes
        for shape_id, details in behavior_dict['shapes'].items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Load connections
        if not connections:
            for connection in behavior_dict['connections']:
                start = QPointF(connection['start'][0], connection['start'][1])
                end = QPointF(connection['end'][0], connection['end'][1])
                conn_type = connection['type']
                print(f"Connecting {start} to {end} with type {conn_type}")
                self.dropArea.connections.append((start, end, conn_type))
        else:
            self.dropArea.connections = connections

        self.dropArea.update()
        print("Finished loading behavior tree")
        


if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())


In [ ]:
#dark theme with imroved anvas and pop up dialog 
import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle, QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox, QMenu, QAction, QFileDialog, QCheckBox)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import shlex
import time
import os
from main_gui import Ui_MainWindow  # Ensure this import matches the file name
import math


class HistoryWidget(QWidget):
    def __init__(self, parent=None):
        super(HistoryWidget, self).__init__(parent)
        self.parent_widget = parent  # Store the reference to the parent widget
        self.setWindowTitle("History Behavior Tree")
        self.setFixedSize(300, 400)
        self.layout = QVBoxLayout(self)

        self.history_list = QListWidget()
        self.history_list.setSelectionMode(QAbstractItemView.MultiSelection)  # Enable multiple selection
        self.history_list.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.layout.addWidget(self.history_list)

        # Add a layout for the top buttons
        self.top_layout = QHBoxLayout()
        self.layout.addLayout(self.top_layout)

        # Add the three-line button
        self.menu_button = QPushButton("≡", self)
        self.menu_button.setFixedSize(30, 30)
        self.menu_button.clicked.connect(self.show_menu)
        self.top_layout.addWidget(self.menu_button)

        # Add the Select button
        self.select_button = QPushButton("Select", self)
        self.select_button.setFixedSize(70, 30)
        self.select_button.clicked.connect(self.select_behavior_tree)  # Connect the Select button
        self.top_layout.addWidget(self.select_button)

        # Add the theme toggle checkbox
        self.themeToggleButton = QCheckBox("Dark Theme")
        self.themeToggleButton.stateChanged.connect(self.parent_widget.toggle_theme)
        self.top_layout.addWidget(self.themeToggleButton)

        self.top_layout.addStretch()

        self.saved_behavior_trees = []  # List to store saved behavior trees

    def show_menu(self):
        menu = QMenu(self)

        save_modify_action = QAction("Save and Modify", self)
        save_compound_action = QAction("Save as Compound", self)
        
        save_modify_action.triggered.connect(self.parent_widget.save_and_modify)
        save_compound_action.triggered.connect(self.parent_widget.save_as_compound)
        
        menu.addAction(save_modify_action)
        menu.addAction(save_compound_action)

        menu.exec_(self.menu_button.mapToGlobal(self.menu_button.rect().bottomLeft()))

    def add_item(self, name, pixmap, behavior_tree):
        item = QListWidgetItem(name)
        item.setData(Qt.UserRole, pixmap)
        item.setData(Qt.UserRole + 1, behavior_tree)  # Store the behavior tree in the item

        # Create a label to show the pixmap
        label = QLabel()
        label.setPixmap(pixmap)
        label.setScaledContents(True)
        label.setFixedSize(280, 280)  # Adjust as needed

        # Add the label to the list item
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(label)
        layout.addStretch()

        item.setSizeHint(widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, widget)

    def select_behavior_tree(self):
        selected_items = self.history_list.selectedItems()
        if not selected_items:
            QMessageBox.warning(self, "Selection Error", "Please select one or more behavior trees from the history.")
            return

        combined_behavior_tree = {}
        combined_connections = []
        offset_y = 0  # Initial offset

        for selected_item in selected_items:
            behavior_tree = selected_item.data(Qt.UserRole + 1)  # Retrieve the stored behavior tree
            combined_behavior_tree, combined_connections = self.combine_trees(combined_behavior_tree, combined_connections, behavior_tree, offset_y)
            offset_y += 200  # Increase offset to stack trees vertically

        print("Combined Behavior Tree:", combined_behavior_tree)
        print("Combined Connections:", combined_connections)
        self.parent_widget.load_behavior_tree(combined_behavior_tree, combined_connections)  # Call the parent's load method

    def combine_trees(self, tree1, connections1, tree2, offset_y):
        combined_tree = tree1.copy()
        combined_connections = connections1.copy()

        # Create a mapping from old IDs to new IDs
        id_mapping = {}

        # Add shapes to the combined tree
        for shape_id, details in tree2.items():
            new_shape_id = f"{shape_id}_{len(combined_tree)}"  # Unique ID
            new_position = QPointF(details['position'][0], details['position'][1] + offset_y)
            combined_tree[new_shape_id] = {
                'type': details['type'],
                'position': (new_position.x(), new_position.y()),
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }
            id_mapping[shape_id] = new_shape_id
            print(f"Added shape {new_shape_id} at position {new_position}")

        # Add connections to the combined tree
        for shape_id, details in tree2.items():
            new_shape_id = id_mapping[shape_id]
            for child in details['children']:
                old_child_id = child['id']
                if old_child_id in id_mapping:
                    new_child_id = id_mapping[old_child_id]
                    start_position = QPointF(combined_tree[new_shape_id]['position'][0], combined_tree[new_shape_id]['position'][1])
                    end_position = QPointF(combined_tree[new_child_id]['position'][0], combined_tree[new_child_id]['position'][1])
                    combined_connections.append((start_position, end_position, child['type']))
                    print(f"Connected {start_position} to {end_position} with type {child['type']}")

        return combined_tree, combined_connections



class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Diamond":
            # Create a custom pixmap for a diamond icon
            pixmap = QPixmap(32, 32)
            pixmap.fill(Qt.transparent)
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))
            painter.setBrush(Qt.gray)
            points = [QPointF(16, 4), QPointF(28, 16), QPointF(16, 28), QPointF(4, 16)]
            painter.drawPolygon(QPolygonF(points))
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(int(position.x() - 20), int(position.y() - 20), 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break


class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        # Initialize ROS core
        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

        # Apply the initial theme
        self.apply_light_theme()

    def apply_dark_theme(self):
        self.load_qss("dark.qss")
        self.dropArea.setStyleSheet("QLabel { background-color : #4B4B4B; }")  # Set to a grey color

    def apply_light_theme(self):
        self.load_qss("light.qss")
        self.dropArea.setStyleSheet("QLabel { background-color : white; }")
        
    def load_qss(self, file_name):
        try:
            qss_file_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), file_name)
            with open(qss_file_path, "r") as f:
                self.setStyleSheet(f.read())
        except FileNotFoundError:
            print(f"QSS file '{file_name}' not found.")
        except Exception as e:
            print(f"Error loading QSS file '{file_name}': {e}")

    def toggle_theme(self, state):
        if state == Qt.Checked:
            self.apply_dark_theme()
        else:
            self.apply_light_theme()


    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        # Access the behavior tree and connections from the drop area
        behavior_tree = self.dropArea.behavior_tree
        connections = self.dropArea.connections

        # Check if there's anything to save
        if not behavior_tree:
            print("No behavior tree to save")
            return

        # Convert behavior tree and connections to dictionaries
        behavior_dict = {
            'shapes': {},
            'connections': []
        }
        for shape_id, details in behavior_tree.items():
            behavior_dict['shapes'][shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        for connection in connections:
            start, end, conn_type = connection
            behavior_dict['connections'].append({
                'start': (start.x(), start.y()),
                'end': (end.x(), end.y()),
                'type': conn_type
            })

        # Save the dictionary as a JSON file
        with open('behavior_tree.json', 'w') as json_file, open('behavior_tree_history.json', 'a') as history_file:
            json.dump(behavior_dict, json_file, indent=4)
            history_file.write(json.dumps(behavior_dict) + "\n")  # Save to history file
        print("Behavior tree saved to behavior_tree.json and appended to history")

        # Create a pixmap of the current drop area
        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)

        # Add to history
        self.historyWidget.add_item("Behavior Tree", pixmap, behavior_dict)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {}
            for shape_id, details in behavior_tree.items():
                behavior_dict[shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")
                self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict, connections=[]):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # Load shapes
        for shape_id, details in behavior_dict['shapes'].items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Load connections
        if not connections:
            for connection in behavior_dict['connections']:
                start = QPointF(connection['start'][0], connection['start'][1])
                end = QPointF(connection['end'][0], connection['end'][1])
                conn_type = connection['type']
                print(f"Connecting {start} to {end} with type {conn_type}")
                self.dropArea.connections.append((start, end, conn_type))
        else:
            self.dropArea.connections = connections

        self.dropArea.update()
        print("Finished loading behavior tree")



if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())


In [ ]:
#dark mode works and even the select button of history tab 
import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle, QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox, QMenu, QAction, QFileDialog, QCheckBox)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import shlex
import time
import os
import math
from main_gui import Ui_MainWindow  # Ensure this import matches the file name


class HistoryWidget(QWidget):
    def __init__(self, parent=None):
        super(HistoryWidget, self).__init__(parent)
        self.parent_widget = parent  # Store the reference to the parent widget
        self.setWindowTitle("History Behavior Tree")
        self.setFixedSize(300, 400)
        self.layout = QVBoxLayout(self)

        self.history_list = QListWidget()
        self.history_list.setSelectionMode(QAbstractItemView.MultiSelection)  # Enable multiple selection
        self.history_list.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.layout.addWidget(self.history_list)

        # Add a layout for the top buttons
        self.top_layout = QHBoxLayout()
        self.layout.addLayout(self.top_layout)

        # Add the three-line button
        self.menu_button = QPushButton("≡", self)
        self.menu_button.setFixedSize(30, 30)
        self.menu_button.clicked.connect(self.show_menu)
        self.top_layout.addWidget(self.menu_button)

        # Add the Select button
        self.select_button = QPushButton("Select", self)
        self.select_button.setFixedSize(70, 30)
        self.select_button.clicked.connect(self.select_behavior_tree)  # Connect the Select button
        self.top_layout.addWidget(self.select_button)

        # Add the theme toggle checkbox
        self.themeToggleButton = QCheckBox("Dark Theme")
        self.themeToggleButton.stateChanged.connect(self.parent_widget.toggle_theme)
        self.top_layout.addWidget(self.themeToggleButton)

        self.top_layout.addStretch()

        self.saved_behavior_trees = []  # List to store saved behavior trees

    def show_menu(self):
        menu = QMenu(self)

        save_modify_action = QAction("Save and Modify", self)
        save_compound_action = QAction("Save as Compound", self)
        
        save_modify_action.triggered.connect(self.parent_widget.save_and_modify)
        save_compound_action.triggered.connect(self.parent_widget.save_as_compound)
        
        menu.addAction(save_modify_action)
        menu.addAction(save_compound_action)

        menu.exec_(self.menu_button.mapToGlobal(self.menu_button.rect().bottomLeft()))

    def add_item(self, name, pixmap, behavior_tree):
        item = QListWidgetItem(name)
        item.setData(Qt.UserRole, pixmap)
        item.setData(Qt.UserRole + 1, behavior_tree)  # Store the behavior tree in the item

        # Create a label to show the pixmap
        label = QLabel()
        label.setPixmap(pixmap)
        label.setScaledContents(True)
        label.setFixedSize(280, 280)  # Adjust as needed

        # Add the label to the list item
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(label)
        layout.addStretch()

        item.setSizeHint(widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, widget)

    def select_behavior_tree(self):
        selected_items = self.history_list.selectedItems()
        if not selected_items:
            QMessageBox.warning(self, "Selection Error", "Please select one or more behavior trees from the history.")
            return

        combined_behavior_tree = {}
        combined_connections = []
        offset_y = 0  # Initial offset

        for selected_item in selected_items:
            behavior_tree = selected_item.data(Qt.UserRole + 1)  # Retrieve the stored behavior tree
            combined_behavior_tree, combined_connections = self.combine_trees(combined_behavior_tree, combined_connections, behavior_tree, offset_y)
            offset_y += 200  # Increase offset to stack trees vertically

        print("Combined Behavior Tree:", combined_behavior_tree)
        print("Combined Connections:", combined_connections)
        self.parent_widget.load_behavior_tree({'shapes': combined_behavior_tree, 'connections': combined_connections})

    def combine_trees(self, tree1, connections1, tree2, offset_y):
        combined_tree = tree1.copy()
        combined_connections = connections1.copy()

        # Create a mapping from old IDs to new IDs
        id_mapping = {}

        # Add shapes to the combined tree
        for shape_id, details in tree2['shapes'].items():
            new_shape_id = f"{shape_id}_{len(combined_tree)}"  # Unique ID
            new_position = QPointF(details['position'][0], details['position'][1] + offset_y)
            combined_tree[new_shape_id] = {
                'type': details['type'],
                'position': (new_position.x(), new_position.y()),
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }
            id_mapping[shape_id] = new_shape_id
            print(f"Added shape {new_shape_id} at position {new_position}")

        # Add connections to the combined tree
        for connection in tree2['connections']:
            start_id = connection['start_id']
            end_id = connection['end_id']
            if start_id in id_mapping and end_id in id_mapping:
                new_start_id = id_mapping[start_id]
                new_end_id = id_mapping[end_id]
                start_position = QPointF(combined_tree[new_start_id]['position'][0], combined_tree[new_start_id]['position'][1])
                end_position = QPointF(combined_tree[new_end_id]['position'][0], combined_tree[new_end_id]['position'][1])
                combined_connections.append({'start': (start_position.x(), start_position.y()), 'end': (end_position.x(), end_position.y()), 'type': connection['type']})
                print(f"Connected {start_position} to {end_position} with type {connection['type']}")

        return combined_tree, combined_connections



class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Diamond":
            # Create a custom pixmap for a diamond icon
            pixmap = QPixmap(32, 32)
            pixmap.fill(Qt.transparent)
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))
            painter.setBrush(Qt.gray)
            points = [QPointF(16, 4), QPointF(28, 16), QPointF(16, 28), QPointF(4, 16)]
            painter.drawPolygon(QPolygonF(points))
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break

    def get_shape_id_at_pos(self, pos):
        for shape, position in self.shapes:
            if self.get_shape_at_pos(pos) == (shape, position):
                return self.get_shape_id(shape, position)
        return None



class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        # Initialize ROS core
        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

        # Apply the initial theme
        self.apply_light_theme()

    def apply_dark_theme(self):
        self.load_qss("dark.qss")
        self.dropArea.setStyleSheet("QLabel { background-color : #4B4B4B; }")  # Set to a grey color

    def apply_light_theme(self):
        self.load_qss("light.qss")
        self.dropArea.setStyleSheet("QLabel { background-color : white; }")
        
    def load_qss(self, file_name):
        try:
            qss_file_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), file_name)
            with open(qss_file_path, "r") as f:
                self.setStyleSheet(f.read())
        except FileNotFoundError:
            print(f"QSS file '{file_name}' not found.")
        except Exception as e:
            print(f"Error loading QSS file '{file_name}': {e}")

    def toggle_theme(self, state):
        if state == Qt.Checked:
            self.apply_dark_theme()
        else:
            self.apply_light_theme()


    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        behavior_tree = self.dropArea.behavior_tree
        connections = self.dropArea.connections

        if not behavior_tree:
            print("No behavior tree to save")
            return

        behavior_dict = {
            'shapes': {},
            'connections': []
        }
        for shape_id, details in behavior_tree.items():
            behavior_dict['shapes'][shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        for connection in connections:
            start, end, conn_type = connection
            start_id = self.dropArea.get_shape_id_at_pos(start)
            end_id = self.dropArea.get_shape_id_at_pos(end)
            if start_id and end_id:
                behavior_dict['connections'].append({
                    'start_id': start_id,
                    'end_id': end_id,
                    'type': conn_type
                })

        with open('behavior_tree.json', 'w') as json_file, open('behavior_tree_history.json', 'a') as history_file:
            json.dump(behavior_dict, json_file, indent=4)
            history_file.write(json.dumps(behavior_dict) + "\n")
        print("Behavior tree saved to behavior_tree.json and appended to history")

        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)
        self.historyWidget.add_item("Behavior Tree", pixmap, behavior_dict)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {}
            for shape_id, details in behavior_tree.items():
                behavior_dict[shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")
                self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict, connections=[]):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # Load shapes
        for shape_id, details in behavior_dict['shapes'].items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Load connections
        if not connections:
            for connection in behavior_dict['connections']:
                start = QPointF(connection['start'][0], connection['start'][1])
                end = QPointF(connection['end'][0], connection['end'][1])
                conn_type = connection['type']
                print(f"Connecting {start} to {end} with type {conn_type}")
                self.dropArea.connections.append((start, end, conn_type))
        else:
            self.dropArea.connections = connections

        self.dropArea.update()
        print("Finished loading behavior tree")




if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())


In [ ]:
#perfect dark mode implementtion with no errors 


import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle, QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox, QMenu, QAction, QFileDialog, QCheckBox)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import shlex
import time
import os
import math
from main_gui import Ui_MainWindow  # Ensure this import matches the file name


class HistoryWidget(QWidget):
    def __init__(self, parent=None):
        super(HistoryWidget, self).__init__(parent)
        self.parent_widget = parent  # Store the reference to the parent widget
        self.setWindowTitle("History Behavior Tree")
        self.setFixedSize(300, 400)
        self.layout = QVBoxLayout(self)

        self.history_list = QListWidget()
        self.history_list.setSelectionMode(QAbstractItemView.MultiSelection)  # Enable multiple selection
        self.history_list.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.layout.addWidget(self.history_list)

        # Add a layout for the top buttons
        self.top_layout = QHBoxLayout()
        self.layout.addLayout(self.top_layout)

        # Add the three-line button
        self.menu_button = QPushButton("≡", self)
        self.menu_button.setFixedSize(30, 30)
        self.menu_button.clicked.connect(self.show_menu)
        self.top_layout.addWidget(self.menu_button)

        # Add the Select button
        self.select_button = QPushButton("Select", self)
        self.select_button.setFixedSize(70, 30)
        self.select_button.clicked.connect(self.select_behavior_tree)  # Connect the Select button
        self.top_layout.addWidget(self.select_button)

        # Add the theme toggle checkbox
        self.themeToggleButton = QCheckBox("Dark Theme")
        self.themeToggleButton.stateChanged.connect(self.parent_widget.toggle_theme)
        self.top_layout.addWidget(self.themeToggleButton)

        self.top_layout.addStretch()

        self.saved_behavior_trees = []  # List to store saved behavior trees

    def show_menu(self):
        menu = QMenu(self)

        save_modify_action = QAction("Save and Modify", self)
        save_compound_action = QAction("Save as Compound", self)
        
        save_modify_action.triggered.connect(self.parent_widget.save_and_modify)
        save_compound_action.triggered.connect(self.parent_widget.save_as_compound)
        
        menu.addAction(save_modify_action)
        menu.addAction(save_compound_action)

        menu.exec_(self.menu_button.mapToGlobal(self.menu_button.rect().bottomLeft()))

    def add_item(self, name, pixmap, behavior_tree):
        item = QListWidgetItem(name)
        item.setData(Qt.UserRole, pixmap)
        item.setData(Qt.UserRole + 1, behavior_tree)  # Store the behavior tree in the item

        # Create a label to show the pixmap
        label = QLabel()
        label.setPixmap(pixmap)
        label.setScaledContents(True)
        label.setFixedSize(280, 280)  # Adjust as needed

        # Add the label to the list item
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(label)
        layout.addStretch()

        item.setSizeHint(widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, widget)

    def select_behavior_tree(self):
        selected_items = self.history_list.selectedItems()
        if not selected_items:
            QMessageBox.warning(self, "Selection Error", "Please select one or more behavior trees from the history.")
            return

        combined_behavior_tree = {}
        combined_connections = []
        offset_y = 0  # Initial offset

        for selected_item in selected_items:
            behavior_tree = selected_item.data(Qt.UserRole + 1)  # Retrieve the stored behavior tree
            combined_behavior_tree, combined_connections = self.combine_trees(combined_behavior_tree, combined_connections, behavior_tree, offset_y)
            offset_y += 200  # Increase offset to stack trees vertically

        print("Combined Behavior Tree:", combined_behavior_tree)
        print("Combined Connections:", combined_connections)
        self.parent_widget.load_behavior_tree({'shapes': combined_behavior_tree, 'connections': combined_connections})

    def combine_trees(self, tree1, connections1, tree2, offset_y):
        combined_tree = tree1.copy()
        combined_connections = connections1.copy()

        # Create a mapping from old IDs to new IDs
        id_mapping = {}

        # Add shapes to the combined tree
        for shape_id, details in tree2['shapes'].items():
            new_shape_id = f"{shape_id}_{len(combined_tree)}"  # Unique ID
            new_position = QPointF(details['position'][0], details['position'][1] + offset_y)
            combined_tree[new_shape_id] = {
                'type': details['type'],
                'position': (new_position.x(), new_position.y()),
                'children': [],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }
            id_mapping[shape_id] = new_shape_id
            print(f"Added shape {new_shape_id} at position {new_position}")

        # Add connections to the combined tree
        for connection in tree2['connections']:
            start_id = connection['start_id']
            end_id = connection['end_id']
            if start_id in id_mapping and end_id in id_mapping:
                new_start_id = id_mapping[start_id]
                new_end_id = id_mapping[end_id]
                start_position = QPointF(combined_tree[new_start_id]['position'][0], combined_tree[new_start_id]['position'][1])
                end_position = QPointF(combined_tree[new_end_id]['position'][0], combined_tree[new_end_id]['position'][1])
                combined_connections.append({
                    'start_id': new_start_id,
                    'end_id': new_end_id,
                    'type': connection['type']
                })
                print(f"Connected {start_position} to {end_position} with type {connection['type']}")

        return combined_tree, combined_connections




class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Diamond":
            # Create a custom pixmap for a diamond icon
            pixmap = QPixmap(32, 32)
            pixmap.fill(Qt.transparent)
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))
            painter.setBrush(Qt.gray)
            points = [QPointF(16, 4), QPointF(28, 16), QPointF(16, 28), QPointF(4, 16)]
            painter.drawPolygon(QPolygonF(points))
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)
        pen = QPen(Qt.black, 2, Qt.SolidLine)
        painter.setPen(pen)

        for shape, position in self.shapes:
            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                          QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                robot_name = details.get('name', '')
                task = details.get('task', '')
                if robot_name:
                    painter.drawText(QPointF(position.x() + 25, position.y()), robot_name)
                if task:
                    painter.drawText(QPointF(position.x() + 25, position.y() + 15), task)

        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        painter.end()

    def draw_arrow(self, painter, start, end, start_shape, end_shape):
        def calculate_intersection(shape, center, direction):
            if shape == "Circle":
                radius = 20  # Circle radius
                return center + direction * radius
            elif shape == "Square":
                half_side = 20  # Half side length of the square
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
            elif shape == "Diamond":
                half_side = 20  # Half side length of the diamond
                direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                if abs(direction.x()) > abs(direction.y()):
                    if direction.x() > 0:
                        return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                else:
                    if direction.y() > 0:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                    else:
                        return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

        # Calculate direction vector
        direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

        # Calculate intersection points
        corrected_start = calculate_intersection(start_shape, start, direction)
        corrected_end = calculate_intersection(end_shape, end, -direction)

        # Draw main line
        line = QLineF(corrected_start, corrected_end)
        painter.drawLine(line)

        # Draw a small circle at the end of the line
        circle_radius = 5  # Radius of the small circle
        painter.setBrush(Qt.black)  # Fill the circle with black color
        painter.drawEllipse(corrected_end, circle_radius, circle_radius)

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break

    def get_shape_id_at_pos(self, pos):
        for shape, position in self.shapes:
            if self.get_shape_at_pos(pos) == (shape, position):
                return self.get_shape_id(shape, position)
        return None



class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        # Initialize ROS core
        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

        # Apply the initial theme
        self.apply_light_theme()

    def apply_dark_theme(self):
        self.load_qss("dark.qss")
        self.dropArea.setStyleSheet("QLabel { background-color : #4B4B4B; }")  # Set to a grey color

    def apply_light_theme(self):
        self.load_qss("light.qss")
        self.dropArea.setStyleSheet("QLabel { background-color : white; }")
        
    def load_qss(self, file_name):
        try:
            qss_file_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), file_name)
            with open(qss_file_path, "r") as f:
                self.setStyleSheet(f.read())
        except FileNotFoundError:
            print(f"QSS file '{file_name}' not found.")
        except Exception as e:
            print(f"Error loading QSS file '{file_name}': {e}")

    def toggle_theme(self, state):
        if state == Qt.Checked:
            self.apply_dark_theme()
        else:
            self.apply_light_theme()


    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        behavior_tree = self.dropArea.behavior_tree
        connections = self.dropArea.connections

        if not behavior_tree:
            print("No behavior tree to save")
            return

        behavior_dict = {
            'shapes': {},
            'connections': []
        }
        for shape_id, details in behavior_tree.items():
            behavior_dict['shapes'][shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        for connection in connections:
            start, end, conn_type = connection
            start_id = self.dropArea.get_shape_id_at_pos(start)
            end_id = self.dropArea.get_shape_id_at_pos(end)
            if start_id and end_id:
                behavior_dict['connections'].append({
                    'start_id': start_id,
                    'end_id': end_id,
                    'type': conn_type
                })

        with open('behavior_tree.json', 'w') as json_file, open('behavior_tree_history.json', 'a') as history_file:
            json.dump(behavior_dict, json_file, indent=4)
            history_file.write(json.dumps(behavior_dict) + "\n")
        print("Behavior tree saved to behavior_tree.json and appended to history")

        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)
        self.historyWidget.add_item("Behavior Tree", pixmap, behavior_dict)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {
                'shapes': {},
                'connections': []
            }
            for shape_id, details in behavior_tree.items():
                behavior_dict['shapes'][shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            for connection in self.dropArea.connections:
                start_id = self.dropArea.get_shape_id_at_pos(connection[0])
                end_id = self.dropArea.get_shape_id_at_pos(connection[1])
                if start_id and end_id:
                    behavior_dict['connections'].append({
                        'start_id': start_id,
                        'end_id': end_id,
                        'type': connection[2]
                    })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")

                # Check if 'shapes' key exists in the behavior_dict
                if 'shapes' in behavior_dict:
                    self.load_behavior_tree(behavior_dict)
                else:
                    # If 'shapes' key does not exist, assume the entire dictionary is the shapes
                    shapes = behavior_dict
                    connections = []  # Assume there are no connections
                    behavior_dict = {'shapes': shapes, 'connections': connections}
                    self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict, connections=[]):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # Load shapes
        for shape_id, details in behavior_dict['shapes'].items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Load connections
        if not connections:
            for connection in behavior_dict['connections']:
                start_id = connection['start_id']
                end_id = connection['end_id']
                start = QPointF(*behavior_dict['shapes'][start_id]['position'])
                end = QPointF(*behavior_dict['shapes'][end_id]['position'])
                conn_type = connection['type']
                print(f"Connecting {start} to {end} with type {conn_type}")
                self.dropArea.connections.append((start, end, conn_type))
        else:
            self.dropArea.connections = connections

        self.dropArea.update()
        print("Finished loading behavior tree")


if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())


   



In [ ]:
#async/sync nodes present, history tab dpoesnt work properly 


import sys
import json
import numpy as np
from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget,
                             QHBoxLayout, QDialog, QDialogButtonBox, QFormLayout, QComboBox, QLineEdit, QPushButton, QToolButton, QStyle, QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox, QMenu, QAction, QFileDialog, QCheckBox)
from PyQt5.QtGui import QPixmap, QPainter, QPen, QDrag, QPolygonF, QIcon, QBrush
from PyQt5.QtCore import Qt, QMimeData, QPointF, QLineF, QRect, QPoint
import subprocess
import shlex
import time
import os
import math
from main_gui import Ui_MainWindow  # Ensure this import matches the file name

ASYNC_COLOR = Qt.green
SYNC_COLOR = Qt.blue


class HistoryWidget(QWidget):
    def __init__(self, parent=None):
        super(HistoryWidget, self).__init__(parent)
        self.parent_widget = parent  # Store the reference to the parent widget
        self.setWindowTitle("History Behavior Tree")
        self.setFixedSize(300, 400)
        self.layout = QVBoxLayout(self)

        self.history_list = QListWidget()
        self.history_list.setSelectionMode(QAbstractItemView.MultiSelection)  # Enable multiple selection
        self.history_list.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.layout.addWidget(self.history_list)

        # Add a layout for the top buttons
        self.top_layout = QHBoxLayout()
        self.layout.addLayout(self.top_layout)

        # Add the three-line button
        self.menu_button = QPushButton("≡", self)
        self.menu_button.setFixedSize(30, 30)
        self.menu_button.clicked.connect(self.show_menu)
        self.top_layout.addWidget(self.menu_button)

        # Add the Select button
        self.select_button = QPushButton("Select", self)
        self.select_button.setFixedSize(70, 30)
        self.select_button.clicked.connect(self.select_behavior_tree)  # Connect the Select button
        self.top_layout.addWidget(self.select_button)

        # Add the theme toggle checkbox
        self.themeToggleButton = QCheckBox("Dark Theme")
        self.themeToggleButton.stateChanged.connect(self.parent_widget.toggle_theme)
        self.top_layout.addWidget(self.themeToggleButton)

        self.top_layout.addStretch()

        self.saved_behavior_trees = []  # List to store saved behavior trees

    def show_menu(self):
        menu = QMenu(self)

        save_modify_action = QAction("Save and Modify", self)
        save_compound_action = QAction("Save as Compound", self)
        
        save_modify_action.triggered.connect(self.parent_widget.save_and_modify)
        save_compound_action.triggered.connect(self.parent_widget.save_as_compound)
        
        menu.addAction(save_modify_action)
        menu.addAction(save_compound_action)

        menu.exec_(self.menu_button.mapToGlobal(self.menu_button.rect().bottomLeft()))

    def add_item(self, name, pixmap, behavior_tree):
        item = QListWidgetItem(name)
        item.setData(Qt.UserRole, pixmap)
        item.setData(Qt.UserRole + 1, behavior_tree)  # Store the behavior tree in the item

        # Create a label to show the pixmap
        label = QLabel()
        label.setPixmap(pixmap)
        label.setScaledContents(True)
        label.setFixedSize(280, 280)  # Adjust as needed

        # Add the label to the list item
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(label)
        layout.addStretch()

        item.setSizeHint(widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, widget)

    def select_behavior_tree(self):
        selected_items = self.history_list.selectedItems()
        if not selected_items:
            QMessageBox.warning(self, "Selection Error", "Please select one or more behavior trees from the history.")
            return

        combined_behavior_tree = {}
        combined_connections = []
        offset_y = 0  # Initial offset

        for selected_item in selected_items:
            behavior_tree = selected_item.data(Qt.UserRole + 1)  # Retrieve the stored behavior tree
            combined_behavior_tree, combined_connections = self.combine_trees(combined_behavior_tree, combined_connections, behavior_tree, offset_y)
            offset_y += 200  # Increase offset to stack trees vertically

        print("Combined Behavior Tree:", combined_behavior_tree)
        print("Combined Connections:", combined_connections)
        self.parent_widget.load_behavior_tree({'shapes': combined_behavior_tree, 'connections': combined_connections})

    def combine_trees(self, tree1, connections1, tree2, offset_y):
        combined_tree = tree1.copy()
        combined_connections = connections1.copy()

        # Create a mapping from old IDs to new IDs
        id_mapping = {}

        # Add shapes to the combined tree
        for shape_id, details in tree2['shapes'].items():
            new_shape_id = f"{shape_id}_{len(combined_tree)}"  # Unique ID
            new_position = QPointF(details['position'][0], details['position'][1] + offset_y)
            combined_tree[new_shape_id] = {
                'type': details['type'],
                'position': (new_position.x(), new_position.y()),
                'children': [],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }
            id_mapping[shape_id] = new_shape_id
            print(f"Added shape {new_shape_id} at position {new_position}")

        # Add connections to the combined tree
        for connection in tree2['connections']:
            start_id = connection['start_id']
            end_id = connection['end_id']
            if start_id in id_mapping and end_id in id_mapping:
                new_start_id = id_mapping[start_id]
                new_end_id = id_mapping[end_id]
                start_position = QPointF(combined_tree[new_start_id]['position'][0], combined_tree[new_start_id]['position'][1])
                end_position = QPointF(combined_tree[new_end_id]['position'][0], combined_tree[new_end_id]['position'][1])
                combined_connections.append({
                    'start_id': new_start_id,
                    'end_id': new_end_id,
                    'type': connection['type']
                })
                print(f"Connected {start_position} to {end_position} with type {connection['type']}")

        return combined_tree, combined_connections




class ShapeButton(QToolButton):
    def __init__(self, shape, parent=None):
        super(ShapeButton, self).__init__(parent)
        self.shape = shape
        self.setFixedSize(50, 50)
        self.setAutoRaise(True)
        self.setToolButtonStyle(Qt.ToolButtonTextUnderIcon)

        if shape == "Circle":
            self.setIcon(self.style().standardIcon(QStyle.SP_DialogYesButton))
        elif shape == "Square":
            # Create a custom pixmap for a square icon
            pixmap = QPixmap(32, 32)  # Size of the pixmap
            pixmap.fill(Qt.transparent)  # Fill the pixmap with transparency
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))  # Set the pen for the border of the square, black with width 2
            painter.setBrush(Qt.gray)  # Fill color of the square
            painter.drawRect(4, 4, 24, 24)  # Draw a square inside the pixmap
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Diamond":
            # Create a custom pixmap for a diamond icon
            pixmap = QPixmap(32, 32)
            pixmap.fill(Qt.transparent)
            painter = QPainter(pixmap)
            painter.setPen(QPen(Qt.black, 2))
            painter.setBrush(Qt.gray)
            points = [QPointF(16, 4), QPointF(28, 16), QPointF(16, 28), QPointF(4, 16)]
            painter.drawPolygon(QPolygonF(points))
            painter.end()
            self.setIcon(QIcon(pixmap))  # Set the custom pixmap as the icon
        elif shape == "Eraser":
            self.setIcon(self.style().standardIcon(QStyle.SP_TrashIcon))

        self.setText(shape)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            mimeData = QMimeData()
            mimeData.setText(self.shape)

            drag = QDrag(self)
            drag.setMimeData(mimeData)

            pixmap = QPixmap(self.size())
            self.render(pixmap)
            drag.setPixmap(pixmap)
            drag.setHotSpot(event.pos())

            drag.exec_(Qt.MoveAction)


class PopupDialog(QDialog):
    def __init__(self, parent=None, popup_type='link'):
        super(PopupDialog, self).__init__(parent)
        self.setWindowTitle("Shape/Link Info")
        self.setFixedSize(300, 200)

        layout = QFormLayout(self)

        if popup_type == 'shape':
            self.nameLabel = QLabel("ROBOT NAME")
            self.nameLineEdit = QLineEdit()
            layout.addRow(self.nameLabel, self.nameLineEdit)

            self.taskLabel = QLabel("TASK")
            self.taskLineEdit = QLineEdit()
            layout.addRow(self.taskLabel, self.taskLineEdit)

            self.descLabel = QLabel("TASK DESCRIPTION")
            self.descLineEdit = QLineEdit()
            layout.addRow(self.descLabel, self.descLineEdit)

        else:
            self.typeLabel = QLabel("TYPE")
            self.typeComboBox = QComboBox()
            self.typeComboBox.addItems(["true", "false", "yes", "no"])
            layout.addRow(self.typeLabel, self.typeComboBox)

        self.buttonBox = QDialogButtonBox(QDialogButtonBox.Save | QDialogButtonBox.Cancel)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)

        layout.addRow(self.buttonBox)

    def get_type(self):
        return self.typeComboBox.currentText() if hasattr(self, 'typeComboBox') else None

    def get_name(self):
        return self.nameLineEdit.text() if hasattr(self, 'nameLineEdit') else None

    def get_task(self):
        return self.taskLineEdit.text() if hasattr(self, 'taskLineEdit') else None

    def get_desc(self):
        return self.descLineEdit.text() if hasattr(self, 'descLineEdit') else None


class DropArea(QLabel):
    def __init__(self, parent=None):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("QLabel { background-color : white; }")
        self.shapes = []
        self.connections = []
        self.last_right_clicked = None
        self.behavior_tree = {}
        self.dragging_shape = None
        self.offset = QPointF()
        self.double_click_timer = None
        self.scale_factor = 1.0
        self.setMouseTracking(True)
        self.panning = False
        self.pan_start = QPoint()
        self.translation = QPointF(0, 0)  # To keep track of translation due to panning

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        position = (event.pos() - self.translation) / self.scale_factor
        shape = event.mimeData().text()

        if shape == "Eraser":
            self.erase_shape(position)
            self.erase_connection(position)
        else:
            if self.dragging_shape:
                self.update_shape_position(self.dragging_shape[0], self.dragging_shape[1], position)
                self.dragging_shape = None
            else:
                self.shapes.append((shape, position))
                shape_id = f"{shape}_{len(self.shapes)}"
                self.show_popup('shape')
                if self.popup_name is not None:
                    self.behavior_tree[shape_id] = {
                        'type': shape,
                        'position': (position.x(), position.y()),
                        'children': [],
                        'name': self.popup_name,
                        'task': self.popup_task,
                        'desc': self.popup_desc
                    }
            self.update()
            event.acceptProposedAction()

    def mousePressEvent(self, event):
        if event.button() == Qt.RightButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                if self.last_right_clicked is None:
                    self.last_right_clicked = clicked_shape
                else:
                    start_shape, start_pos = self.last_right_clicked
                    end_shape, end_pos = clicked_shape
                    self.show_popup('link')
                    if self.popup_type:
                        self.connections.append((start_pos, end_pos, self.popup_type))
                        start_shape_id = self.get_shape_id(start_shape, start_pos)
                        end_shape_id = self.get_shape_id(end_shape, end_pos)
                        if start_shape_id and end_shape_id:
                            self.behavior_tree[start_shape_id]['children'].append({
                                'id': end_shape_id,
                                'type': self.popup_type
                            })
                    self.last_right_clicked = None
                    self.update()
        elif event.button() == Qt.LeftButton:
            clicked_pos = (event.pos() - self.translation) / self.scale_factor
            clicked_shape = self.get_shape_at_pos(clicked_pos)
            if clicked_shape:
                self.dragging_shape = clicked_shape
                shape, position = clicked_shape
                self.offset = position - clicked_pos
            else:
                self.double_click_timer = self.startTimer(200)
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ClosedHandCursor)
            self.pan_start = event.pos()
            self.panning = True

    def mouseMoveEvent(self, event):
        if self.panning:
            delta = event.pos() - self.pan_start
            self.pan_start = event.pos()
            self.translation += delta
            self.update()
        elif self.dragging_shape:
            new_pos = (event.pos() - self.translation) / self.scale_factor + self.offset
            shape, old_pos = self.dragging_shape
            self.update_shape_position(shape, old_pos, new_pos)
            self.dragging_shape = (shape, new_pos)
            self.update()  # Ensure this line is present to continuously redraw the arrows
        else:
            super(DropArea, self).mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.dragging_shape:
                self.dragging_shape = None
        elif event.button() == Qt.MiddleButton:
            self.setCursor(Qt.ArrowCursor)
            self.panning = False

    def mouseDoubleClickEvent(self, event):
        if self.double_click_timer:
            self.killTimer(self.double_click_timer)
            self.double_click_timer = None
        clicked_pos = (event.pos() - self.translation) / self.scale_factor
        clicked_shape = self.get_shape_at_pos(clicked_pos)
        if clicked_shape:
            self.edit_shape(clicked_shape)

    def edit_shape(self, shape_info):
        shape, position = shape_info
        shape_id = self.get_shape_id(shape, position)
        if shape_id:
            details = self.behavior_tree[shape_id]
            self.show_popup('shape', details)
            if self.popup_name is not None:
                details['name'] = self.popup_name
                details['task'] = self.popup_task
                details['desc'] = self.popup_desc
                self.update()        

    def show_popup(self, popup_type='link', current_details=None):
        dialog = PopupDialog(self, popup_type)
        if current_details:
            if popup_type == 'shape':
                dialog.nameLineEdit.setText(current_details.get('name', ''))
                dialog.taskLineEdit.setText(current_details.get('task', ''))
                dialog.descLineEdit.setText(current_details.get('desc', ''))
        if dialog.exec_() == QDialog.Accepted:
            self.popup_type = dialog.get_type()
            self.popup_name = dialog.get_name() if popup_type == 'shape' else None
            self.popup_task = dialog.get_task() if popup_type == 'shape' else None
            self.popup_desc = dialog.get_desc() if popup_type == 'shape' else None
        else:
            self.popup_type = None
            self.popup_name = None
            self.popup_task = None
            self.popup_desc = None

    def timerEvent(self, event):
        self.killTimer(event.timerId())
        self.double_click_timer = None

    def wheelEvent(self, event):
        cursor_pos = event.pos()
        old_scale_factor = self.scale_factor
        if event.angleDelta().y() > 0:
            self.scale_factor *= 1.1
        else:
            self.scale_factor /= 1.1
        scale_ratio = self.scale_factor / old_scale_factor

        # Adjust translation to keep the cursor at the same relative position
        self.translation = cursor_pos - (cursor_pos - self.translation) * scale_ratio

        self.update()

    def get_shape_at_pos(self, pos):
        for shape, position in self.shapes:
            if shape == "Circle" and (position - pos).manhattanLength() < 25:
                return (shape, position)
            elif shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25:
                return (shape, position)
            elif shape == "Diamond" and self.is_point_in_diamond(pos, position):
                return (shape, position)
        return None

    def is_point_in_diamond(self, point, center):
        diamond_size = 20  # Half size of the diamond's bounding box
        dx = abs(point.x() - center.x())
        dy = abs(point.y() - center.y())
        return (dx + dy) <= diamond_size

    def update_shape_position(self, shape, old_position, new_position):
        for i, (s, pos) in enumerate(self.shapes):
            if s == shape and pos == old_position:
                self.shapes[i] = (shape, new_position)
                shape_id = self.get_shape_id(shape, old_position)
                if shape_id:
                    self.behavior_tree[shape_id]['position'] = (new_position.x(), new_position.y())
                break

    def get_child_conditions(self, parent_id):
        if parent_id is None:
            return {}

        child_conditions = {}
        for child in self.behavior_tree[parent_id]['children']:
            child_id = child['id']
            for connection in self.connections:
                start_id = self.get_shape_id_at_pos(connection[0])
                end_id = self.get_shape_id_at_pos(connection[1])
                if start_id == parent_id and end_id == child_id:
                    condition = connection[2]
                    if condition in child_conditions:
                        child_conditions[condition].append(child_id)
                    else:
                        child_conditions[condition] = [child_id]
        return child_conditions

    def paintEvent(self, event):
        super(DropArea, self).paintEvent(event)
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setRenderHint(QPainter.SmoothPixmapTransform)
        painter.translate(self.translation)
        painter.scale(self.scale_factor, self.scale_factor)

        # First, draw the connections (links) between nodes
        for connection in self.connections:
            start, end, conn_type = connection
            start_shape = self.get_shape_at_pos(start)
            end_shape = self.get_shape_at_pos(end)
            if start_shape and end_shape:
                self.draw_arrow(painter, start, end, start_shape[0], end_shape[0])
                self.draw_text(painter, start, end, conn_type)

        # Then, draw the nodes
        for shape, position in self.shapes:
            shape_id = self.get_shape_id(shape, position)
            if shape_id:
                details = self.behavior_tree[shape_id]
                parent_id = self.get_parent_id(shape_id)
                siblings = self.get_siblings(parent_id)
                child_conditions = self.get_child_conditions(parent_id)

                if parent_id is None or len(siblings) <= 1:
                    brush = QBrush(SYNC_COLOR)
                    self.setToolTip("Synchronous Node")
                else:
                    is_async = False
                    for condition, child_ids in child_conditions.items():
                        if shape_id in child_ids and len(child_ids) > 1:
                            is_async = True
                            break

                    if is_async:
                        brush = QBrush(ASYNC_COLOR)
                        self.setToolTip("Asynchronous Node")
                    else:
                        brush = QBrush(SYNC_COLOR)
                        self.setToolTip("Synchronous Node")
                painter.setBrush(brush)

            if shape == "Circle":
                painter.drawEllipse(position, 20, 20)
            elif shape == "Square":
                painter.drawRect(position.x() - 20, position.y() - 20, 40, 40)
            elif shape == "Diamond":
                points = [QPointF(position.x(), position.y() - 20), QPointF(position.x() + 20, position.y()),
                        QPointF(position.x(), position.y() + 20), QPointF(position.x() - 20, position.y())]
                painter.drawPolygon(QPolygonF(points))

        painter.end()



    def get_parent_id(self, shape_id):
        for parent_id, details in self.behavior_tree.items():
            for child in details['children']:
                if child['id'] == shape_id:
                    return parent_id
        return None

    def get_siblings(self, parent_id):
        if parent_id is None:
            return []
        return [child['id'] for child in self.behavior_tree[parent_id]['children']]
    
    def get_connections_of_node(self, node_id):
        connections = []
        for connection in self.connections:
            start_id = self.get_shape_id_at_pos(connection[0])
            end_id = self.get_shape_id_at_pos(connection[1])
            if start_id == node_id or end_id == node_id:
                connections.append(connection)
        return connections




    def get_connections_of_node(self, node_id):
            connections = []
            for connection in self.connections:
                start_id = self.get_shape_id_at_pos(connection[0])
                end_id = self.get_shape_id_at_pos(connection[1])
                if start_id == node_id or end_id == node_id:
                    connections.append(connection)
            return connections


    def draw_arrow(self, painter, start, end, start_shape, end_shape):
            def calculate_intersection(shape, center, direction):
                if shape == "Circle":
                    radius = 20  # Circle radius
                    return center + direction * radius
                elif shape == "Square":
                    half_side = 20  # Half side length of the square
                    direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                    if abs(direction.x()) > abs(direction.y()):
                        if direction.x() > 0:
                            return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                        else:
                            return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        if direction.y() > 0:
                            return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                        else:
                            return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)
                elif shape == "Diamond":
                    half_side = 20  # Half side length of the diamond
                    direction = direction / np.linalg.norm([direction.x(), direction.y()])  # Normalize direction
                    if abs(direction.x()) > abs(direction.y()):
                        if direction.x() > 0:
                            return center + QPointF(half_side, half_side * direction.y() / abs(direction.x()))
                        else:
                            return center + QPointF(-half_side, half_side * direction.y() / abs(direction.x()))
                    else:
                        if direction.y() > 0:
                            return center + QPointF(half_side * direction.x() / abs(direction.y()), half_side)
                        else:
                            return center + QPointF(half_side * direction.x() / abs(direction.y()), -half_side)

            # Calculate direction vector
            direction = (end - start) / np.linalg.norm([end.x() - start.x(), end.y() - start.y()])

            # Calculate intersection points
            corrected_start = calculate_intersection(start_shape, start, direction)
            corrected_end = calculate_intersection(end_shape, end, -direction)

            # Draw main line
            line = QLineF(corrected_start, corrected_end)
            painter.drawLine(line)

            # Draw a small circle at the end of the line
            circle_radius = 5  # Radius of the small circle
            painter.setBrush(Qt.black)  # Fill the circle with black color
            painter.drawEllipse(corrected_end, circle_radius, circle_radius)   

    def draw_text(self, painter, start, end, text):
        mid_point = (start + end) / 2
        painter.drawText(mid_point, text)

    def resizeEvent(self, event):
        new_size = event.size()
        old_size = event.oldSize()

        if old_size.width() < 0 or old_size.height() < 0:
            old_size = new_size

        self.translation += QPointF(
            (new_size.width() - old_size.width()) / 2,
            (new_size.height() - old_size.height()) / 2
        )

        super(DropArea, self).resizeEvent(event)

    def get_shape_id(self, shape, position):
        for shape_id, details in self.behavior_tree.items():
            if details['type'] == shape and details['position'] == (position.x(), position.y()):
                return shape_id
        return None

    def erase_shape(self, pos):
        for shape, position in self.shapes:
            if (shape == "Circle" and (position - pos).manhattanLength() < 25) or \
                    (shape == "Square" and abs(position.x() - pos.x()) < 25 and abs(position.y() - pos.y()) < 25) or \
                    (shape == "Diamond" and self.is_point_in_diamond(pos, position)):
                self.shapes.remove((shape, position))
                shape_id = self.get_shape_id(shape, position)
                if shape_id:
                    del self.behavior_tree[shape_id]
                    self.connections = [conn for conn in self.connections if conn[0] != position and conn[1] != position]
                self.update()
                break

    def erase_connection(self, pos):
        for connection in self.connections:
            start, end, conn_type = connection
            line = QLineF(start, end)
            if line.p1().x() <= pos.x() <= line.p2().x() or line.p2().x() <= pos.x() <= line.p1().x():
                if line.p1().y() <= pos.y() <= line.p2().y() or line.p2().y() <= pos.y() <= line.p1().y():
                    self.connections.remove(connection)
                    self.update()
                    break

    def get_shape_id_at_pos(self, pos):
        for shape, position in self.shapes:
            if self.get_shape_at_pos(pos) == (shape, position):
                return self.get_shape_id(shape, position)
        return None



class RPLLineGUI(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super(RPLLineGUI, self).__init__()
        self.setupUi(self)

        # Initialize ROS core
        self.roscore = None
        try:
            self.roscore = subprocess.Popen(shlex.split('roscore'))
            time.sleep(5)  # Wait for roscore to start
        except subprocess.CalledProcessError as e:
            print("Error starting roscore:", e)

        # Connect buttons to methods
        self.loadView.clicked.connect(self.load_new_view)
        self.saveView.clicked.connect(self.save_view_config)
        self.newSensorView.clicked.connect(self.new_sensor_view)
        self.refreshSensors.clicked.connect(self.refresh_sensors)
        self.logPose.clicked.connect(self.log_pose)
        self.setReference.clicked.connect(self.set_reference)
        self.editPoses.clicked.connect(self.edit_poses)
        self.runButton.clicked.connect(self.test_motion)
        self.saveAllMotions.clicked.connect(self.save_motions)
        self.loadButton.clicked.connect(self.load_motion)
        self.addMotionBttn.clicked.connect(self.add_motion)
        self.deleteMotion.clicked.connect(self.delete_motion)

        # Add logo to the top left corner
        self.logoLabel = QLabel(self)
        self.logoPixmap = QPixmap("/home/nmis/Downloads/nmis-logo-full-colour-e1657196524147.png")  # Replace with the path to your logo
        self.logoLabel.setPixmap(self.logoPixmap)
        self.logoLabel.setAlignment(Qt.AlignCenter)

        # Adjust logo size
        logo_size = self.logoPixmap.size()
        self.logoLabel.setFixedSize(logo_size)

        # Add drag-and-drop shapes to the sidebar
        self.circleButton = ShapeButton("Circle", self)
        self.squareButton = ShapeButton("Square", self)
        self.diamondButton = ShapeButton("Diamond", self)
        self.eraserButton = ShapeButton("Eraser", self)
        self.shapeLayout = QVBoxLayout()
        self.shapeLayout.addWidget(self.circleButton)
        self.shapeLayout.addWidget(self.squareButton)
        self.shapeLayout.addWidget(self.diamondButton)
        self.shapeLayout.addWidget(self.eraserButton)

        self.shapeContainer = QWidget()
        self.shapeContainer.setLayout(self.shapeLayout)

        self.sidebarLayout = QVBoxLayout()
        self.sidebarLayout.addWidget(self.logoLabel)  # Add logo to the sidebar
        self.sidebarLayout.addWidget(self.shapeContainer)
        self.sidebarLayout.addStretch()  # Add stretch to push the save button to the bottom

        # Add Save button
        
        self.saveButton = QPushButton("Save Behavior Tree", self)
        self.saveButton.clicked.connect(self.save_behavior_tree)
        self.sidebarLayout.addWidget(self.saveButton)

        # Add Open button
        self.openButton = QPushButton("Open Behavior Tree", self)
        self.openButton.clicked.connect(self.open_behavior_tree)
        self.sidebarLayout.addWidget(self.openButton)

        self.sidebar = QWidget()
        self.sidebar.setLayout(self.sidebarLayout)

        # Initialize DropArea
        self.dropArea = DropArea(self)

        # Initialize History Widget
        self.historyWidget = HistoryWidget(self)

        # Main layout
        self.mainLayout = QHBoxLayout()
        self.mainLayout.addWidget(self.sidebar)
        self.mainLayout.addWidget(self.dropArea)
        self.mainLayout.addWidget(self.historyWidget)

        # Create a central widget and set the main layout
        central_widget = QWidget()
        central_widget.setLayout(self.mainLayout)
        self.setCentralWidget(central_widget)

        # Apply the initial theme
        self.apply_light_theme()

    def apply_dark_theme(self):
        self.load_qss("dark.qss")
        self.dropArea.setStyleSheet("QLabel { background-color : #4B4B4B; }")  # Set to a grey color

    def apply_light_theme(self):
        self.load_qss("light.qss")
        self.dropArea.setStyleSheet("QLabel { background-color : white; }")
        
    def load_qss(self, file_name):
        try:
            qss_file_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), file_name)
            with open(qss_file_path, "r") as f:
                self.setStyleSheet(f.read())
        except FileNotFoundError:
            print(f"QSS file '{file_name}' not found.")
        except Exception as e:
            print(f"Error loading QSS file '{file_name}': {e}")

    def toggle_theme(self, state):
        if state == Qt.Checked:
            self.apply_dark_theme()
        else:
            self.apply_light_theme()


    def load_new_view(self):
        pass

    def save_view_config(self):
        pass

    def new_sensor_view(self):
        pass

    def refresh_sensors(self):
        pass

    def log_pose(self):
        pass

    def set_reference(self):
        pass

    def edit_poses(self):
        pass

    def test_motion(self):
        pass

    def save_motions(self):
        pass

    def load_motion(self):
        pass

    def add_motion(self):
        pass

    def delete_motion(self):
        pass

    def save_behavior_tree(self):
        print("Save button clicked")
        behavior_tree = self.dropArea.behavior_tree
        connections = self.dropArea.connections

        if not behavior_tree:
            print("No behavior tree to save")
            return

        behavior_dict = {
            'shapes': {},
            'connections': []
        }
        for shape_id, details in behavior_tree.items():
            behavior_dict['shapes'][shape_id] = {
                'type': details['type'],
                'position': details['position'],
                'children': details['children'],
                'name': details['name'],
                'task': details['task'],
                'desc': details['desc']
            }

        for connection in connections:
            start, end, conn_type = connection
            start_id = self.dropArea.get_shape_id_at_pos(start)
            end_id = self.dropArea.get_shape_id_at_pos(end)
            if start_id and end_id:
                behavior_dict['connections'].append({
                    'start_id': start_id,
                    'end_id': end_id,
                    'type': conn_type
                })

        with open('behavior_tree.json', 'w') as json_file, open('behavior_tree_history.json', 'a') as history_file:
            json.dump(behavior_dict, json_file, indent=4)
            history_file.write(json.dumps(behavior_dict) + "\n")
        print("Behavior tree saved to behavior_tree.json and appended to history")

        pixmap = QPixmap(self.dropArea.size())
        self.dropArea.render(pixmap)
        self.historyWidget.add_item("Behavior Tree", pixmap, behavior_dict)

    def save_and_modify(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Convert behavior tree to a dictionary
            behavior_dict = {
                'shapes': {},
                'connections': []
            }
            for shape_id, details in behavior_tree.items():
                behavior_dict['shapes'][shape_id] = {
                    'type': details['type'],
                    'position': details['position'],
                    'children': details['children'],
                    'name': details['name'],
                    'task': details['task'],
                    'desc': details['desc']
                }

            for connection in self.dropArea.connections:
                start_id = self.dropArea.get_shape_id_at_pos(connection[0])
                end_id = self.dropArea.get_shape_id_at_pos(connection[1])
                if start_id and end_id:
                    behavior_dict['connections'].append({
                        'start_id': start_id,
                        'end_id': end_id,
                        'type': connection[2]
                    })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump(behavior_dict, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")

    def save_as_compound(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getSaveFileName(self, "Save Behavior Tree as Compound", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Saving compound behavior tree to: {file_name}")

            # Access the behavior tree from the drop area
            behavior_tree = self.dropArea.behavior_tree

            # Check if there's anything to save
            if not behavior_tree:
                print("No behavior tree to save")
                return

            # Combine all details into one compound node
            combined_details = {
                'type': 'Diamond',
                'position': (self.dropArea.width() / 2, self.dropArea.height() / 2),
                'children': [],
                'name': 'Compound Node',
                'task': 'Combined Task',
                'desc': 'This is a combined node of all tasks'
            }

            for shape_id, details in behavior_tree.items():
                combined_details['children'].append({
                    'id': shape_id,
                    'type': 'child'
                })

            # Save the dictionary as a JSON file
            with open(file_name, 'w') as json_file:
                json.dump({'compound_node': combined_details}, json_file, indent=4)
            print(f"Behavior tree saved to {file_name}")
        else:
            print("Save as Compound operation was canceled")

    def open_behavior_tree(self):
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Behavior Tree", "", "JSON Files (*.json);;All Files (*)", options=options)
        if file_name:
            print(f"Loading behavior tree from: {file_name}")
            with open(file_name, 'r') as json_file:
                behavior_dict = json.load(json_file)
                print(f"Behavior tree loaded: {behavior_dict}")

                # Check if 'shapes' key exists in the behavior_dict
                if 'shapes' in behavior_dict:
                    self.load_behavior_tree(behavior_dict)
                else:
                    # If 'shapes' key does not exist, assume the entire dictionary is the shapes
                    shapes = behavior_dict
                    connections = []  # Assume there are no connections
                    behavior_dict = {'shapes': shapes, 'connections': connections}
                    self.load_behavior_tree(behavior_dict)
        else:
            print("Open operation was canceled")

    def load_behavior_tree(self, behavior_dict, connections=[]):
        self.dropArea.shapes.clear()
        self.dropArea.connections.clear()
        self.dropArea.behavior_tree.clear()

        print("Loading shapes and connections into the canvas")
        # Load shapes
        for shape_id, details in behavior_dict['shapes'].items():
            if 'position' in details:
                position = QPointF(details['position'][0], details['position'][1])
                shape = details['type']
                print(f"Loading shape {shape} at position {position}")
                self.dropArea.shapes.append((shape, position))
                self.dropArea.behavior_tree[shape_id] = {
                    'type': shape,
                    'position': (position.x(), position.y()),
                    'children': details.get('children', []),
                    'name': details.get('name', ''),
                    'task': details.get('task', ''),
                    'desc': details.get('desc', '')
                }
            else:
                print(f"Error: 'position' key not found in shape details for {shape_id}")

        # Load connections
        if not connections:
            for connection in behavior_dict['connections']:
                start_id = connection['start_id']
                end_id = connection['end_id']
                start = QPointF(*behavior_dict['shapes'][start_id]['position'])
                end = QPointF(*behavior_dict['shapes'][end_id]['position'])
                conn_type = connection['type']
                print(f"Connecting {start} to {end} with type {conn_type}")
                self.dropArea.connections.append((start, end, conn_type))
        else:
            self.dropArea.connections = connections

        self.dropArea.update()
        print("Finished loading behavior tree")


if __name__ == "__main__":
    print("Starting application...")
    app = QApplication(sys.argv)
    window = RPLLineGUI()
    window.show()
    print("Application running...")
    sys.exit(app.exec_())

